## BUS DETAILS

In [1]:
from selenium import webdriver
from selenium.webdriver import ActionChains
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.common.exceptions import TimeoutException, NoSuchElementException
import pandas as pd
import time
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

## KERALA

In [2]:
# Load the CSV file into a DataFrame
df = pd.read_csv("scraped bus details/df.kerala.csv")

# Display the DataFrame
df



,Route_name,Route_link
0,Bangalore to Kozhikode,https://www.redbus.in/bus-tickets/bangalore-to...
1,Kozhikode to Ernakulam,https://www.redbus.in/bus-tickets/kozhikode-to...
2,Kozhikode to Bangalore,https://www.redbus.in/bus-tickets/kozhikode-to...
3,Ernakulam to Kozhikode,https://www.redbus.in/bus-tickets/ernakulam-to...
4,Mysore to Kozhikode,https://www.redbus.in/bus-tickets/mysore-to-ko...
5,Kozhikode to Mysore,https://www.redbus.in/bus-tickets/kozhikode-to...
6,Kozhikode to Thrissur,https://www.redbus.in/bus-tickets/kozhikode-to...
7,Kozhikode to Thiruvananthapuram,https://www.redbus.in/bus-tickets/kozhikode-to...
8,Kozhikode to Kottayam,https://www.redbus.in/bus-tickets/kozhikode-to...
9,Thrissur to Kozhikode,https://www.redbus.in/bus-tickets/thrissur-to-...


In [3]:
driver_k = webdriver.Chrome()  # Initialize the Chrome WebDriver
Bus_names_k = []  # List to store bus names
Bus_types_k = []  # List to store bus types
Departure_k = []  # List to store departure times
Arrival_k = []  # List to store arrival times
Ratings_k = []  # List to store ratings
Total_Duration_k = []  # List to store total travel duration
Prices_k = []  # List to store prices
Seats_Available_k = []  # List to store seat availability
Route_names = []  # List to store route names
Route_links = []  # List to store route links

# Loop through each row of the DataFrame df
for i, r in df.iterrows():  
    link = r["Route_link"]  # Get the route link from the DataFrame
    routes = r["Route_name"]  # Get the route name from the DataFrame

    # Open the route link in the browser
    driver_k.get(link)  
    time.sleep(2)  # Wait for 2 seconds for the page to load

    # Find clickable elements that contain the route link
    elements = driver_k.find_elements(By.XPATH, f"//a[contains(@href, '{link}')]")  
    for element in elements:
        element.click()  # Click the element to reveal bus details
        time.sleep(2)  # Wait for 2 seconds for the page to update

    # Try clicking the "View Buses" button if available
    try:
        clicks = driver_k.find_element(By.XPATH, "//div[@class='button']")
        clicks.click()  # Click the button to view buses
    except:
        continue  # Skip to the next link if the button is not found
    time.sleep(2)  # Wait for the page to update

    scrolling = True  # Initialize a flag to start scrolling
    while scrolling:
        old_page_source = driver_k.page_source  # Store the current page source

        # Scroll down the page using the PAGE_DOWN key
        ActionChains(driver_k).send_keys(Keys.PAGE_DOWN).perform()  
        time.sleep(5)  # Wait for 5 seconds for the page to load more content

        new_page_source = driver_k.page_source  # Get the new page source

        # Stop scrolling if no new content is loaded
        if new_page_source == old_page_source:
            scrolling = False

    # Extract bus details such as name, type, departure time, etc.
    bus_name = driver_k.find_elements(By.XPATH, "//div[@class='travels lh-24 f-bold d-color']")
    bus_type = driver_k.find_elements(By.XPATH, "//div[@class='bus-type f-12 m-top-16 l-color evBus']")
    start_time = driver_k.find_elements(By.XPATH, "//*[@class='dp-time f-19 d-color f-bold']")
    end_time = driver_k.find_elements(By.XPATH, "//*[@class='bp-time f-19 d-color disp-Inline']")
    total_duration = driver_k.find_elements(By.XPATH, "//*[@class='dur l-color lh-24']")
    price = driver_k.find_elements(By.XPATH, '//div[@class="fare d-block"]//span')
    seats = driver_k.find_elements(By.XPATH, "//div[contains(@class, 'seat-left')]")

    # Attempt to extract ratings (if available)
    try:
        rating = driver_k.find_elements(By.XPATH, "//div[@class='clearfix row-one']/div[@class='column-six p-right-10 w-10 fl']")
    except:
        continue  # Skip to the next link if no ratings are found

    # Append extracted details to respective lists
    for bus in bus_name:
        Bus_names_k.append(bus.text)  # Append bus name to the list
        Route_links.append(link)  # Append route link to the list
        Route_names.append(routes)  # Append route name to the list
    for bus_type_elem in bus_type:
        Bus_types_k.append(bus_type_elem.text)  # Append bus type to the list
    for start_time_elem in start_time:
        Departure_k.append(start_time_elem.text)  # Append departure time to the list
    for end_time_elem in end_time:
        Arrival_k.append(end_time_elem.text)  # Append arrival time to the list
    for total_duration_elem in total_duration:
        Total_Duration_k.append(total_duration_elem.text)  # Append total duration to the list
    for ratings_elem in rating:
        Ratings_k.append(ratings_elem.text)  # Append ratings to the list
    for price_elem in price:
        Prices_k.append(price_elem.text)  # Append price to the list
    for seats_elem in seats:
        Seats_Available_k.append(seats_elem.text)  # Append available seats to the list

# Indicate successful completion
print("Successfully Completed")


Successfully Completed


In [4]:
# Create a dictionary where each key corresponds to a column name, and the value is the list of bus details
data = {
    'Bus_name': Bus_names_k,  # List containing the names of buses
    'Bus_type': Bus_types_k,  # List containing the types of buses (e.g., Sleeper, AC, etc.)
    'Start_time': Departure_k,  # List containing the departure times of the buses
    'End_time': Arrival_k,  # List containing the arrival times of the buses
    'Total_duration': Total_Duration_k,  # List containing the total travel duration for each bus
    'Price': Prices_k,  # List containing the price of each bus ticket
    "Seats_Available": Seats_Available_k,  # List containing the number of seats available for each bus
    "Ratings": Ratings_k,  # List containing the ratings for each bus
    'Route_name': Route_names,  # List containing the name of the routes (e.g., "Kozhikode to Ernakulam")
    'Route_link': Route_links  # List containing the URL links for each route
}

# Create a pandas DataFrame from the dictionary 'data' where each key-value pair becomes a column in the DataFrame
df_kerala = pd.DataFrame(data)

# Define the path where the CSV file will be saved
path = r"/Users/priyankamalavade/Desktop/redbus_scraping/scraped_all_required_bus_details/df_kerala.csv"


# Convert the DataFrame 'df_kerala' into a CSV file and save it at the specified path
# index=False ensures that the row index is not included in the CSV file
df_kerala.to_csv(path, index=False)

# Print a message indicating that the process has been completed successfully
print("Successfully Completed")


Successfully Completed


In [5]:
df_kerala

,Bus_name,Bus_type,Start_time,End_time,Total_duration,Price,Seats_Available,Ratings,Route_name,Route_link
0,MMK Travels,A/C Seater / Sleeper (2+1),22:30,07:30,09h 00m,750,24 Seats available,4.6\n556,Bangalore to Kozhikode,https://www.redbus.in/bus-tickets/bangalore-to...
1,P K Travels,A/C Seater / Sleeper (2+1),21:30,06:00,08h 30m,684,23 Seats available,4.2\n62,Bangalore to Kozhikode,https://www.redbus.in/bus-tickets/bangalore-to...
2,P K Travels,A/C Sleeper (2+1),18:45,06:30,11h 45m,810,24 Seats available,4.1\n110,Bangalore to Kozhikode,https://www.redbus.in/bus-tickets/bangalore-to...
3,AdSAAM BUS,VE A/C Sleeper (2+1),21:45,06:45,09h 00m,786,17 Seats available,4.1\n289,Bangalore to Kozhikode,https://www.redbus.in/bus-tickets/bangalore-to...
4,NS Transports,Non A/C Seater / Sleeper (2+1),21:45,06:10,08h 25m,699,19 Seats available,4.2\n276,Bangalore to Kozhikode,https://www.redbus.in/bus-tickets/bangalore-to...
...,...,...,...,...,...,...,...,...,...,...
174,,,,,,,,,Kozhikode to Aluva,https://www.redbus.in/bus-tickets/kozhikode-to...
175,,,,,,,,,Kozhikode to Aluva,https://www.redbus.in/bus-tickets/kozhikode-to...
176,,,,,,,,,Kozhikode to Aluva,https://www.redbus.in/bus-tickets/kozhikode-to...
177,,,,,,,,,Kozhikode to Aluva,https://www.redbus.in/bus-tickets/kozhikode-to...


## ANDRA

In [15]:
# Load the CSV file into a DataFrame
df = pd.read_csv("scraped bus details/df.andhra.csv")

# Display the DataFrame
df

,Route_name,Route_link
0,Hyderabad to Vijayawada,https://www.redbus.in/bus-tickets/hyderabad-to...
1,Vijayawada to Hyderabad,https://www.redbus.in/bus-tickets/vijayawada-t...
2,Kakinada to Visakhapatnam,https://www.redbus.in/bus-tickets/kakinada-to-...
3,Chittoor (Andhra Pradesh) to Bangalore,https://www.redbus.in/bus-tickets/chittoor-and...
4,Bangalore to Tirupati,https://www.redbus.in/bus-tickets/bangalore-to...
5,Tirupati to Bangalore,https://www.redbus.in/bus-tickets/tirupathi-to...
6,Kadapa to Bangalore,https://www.redbus.in/bus-tickets/kadapa-to-ba...
7,Hyderabad to Ongole,https://www.redbus.in/bus-tickets/hyderabad-to...
8,Anantapur (andhra pradesh) to Bangalore,https://www.redbus.in/bus-tickets/ananthapur-t...
9,Bangalore to Kadapa,https://www.redbus.in/bus-tickets/bangalore-to...


In [19]:
driver_k = webdriver.Chrome()  # Initialize the Chrome WebDriver
Bus_names_k = []  # List to store bus names
Bus_types_k = []  # List to store bus types
Departure_k = []  # List to store departure times
Arrival_k = []  # List to store arrival times
Ratings_k = []  # List to store ratings
Total_Duration_k = []  # List to store total travel duration
Prices_k = []  # List to store prices
Seats_Available_k = []  # List to store seat availability
Route_names = []  # List to store route names
Route_links = []  # List to store route links

# Loop through each row of the DataFrame df
for i, r in df.iterrows():  
    link = r["Route_link"]  # Get the route link from the DataFrame
    routes = r["Route_name"]  # Get the route name from the DataFrame

    # Open the route link in the browser
    driver_k.get(link)  
    time.sleep(2)  # Wait for 2 seconds for the page to load

    # Find clickable elements that contain the route link
    elements = driver_k.find_elements(By.XPATH, f"//a[contains(@href, '{link}')]")  
    for element in elements:
        element.click()  # Click the element to reveal bus details
        time.sleep(2)  # Wait for 2 seconds for the page to update

    # Try clicking the "View Buses" button if available
    try:
        clicks = driver_k.find_element(By.XPATH, "//div[@class='button']")
        clicks.click()  # Click the button to view buses
    except:
        continue  # Skip to the next link if the button is not found
    time.sleep(2)  # Wait for the page to update

    scrolling = True  # Initialize a flag to start scrolling
    while scrolling:
        old_page_source = driver_k.page_source  # Store the current page source

        # Scroll down the page using the PAGE_DOWN key
        ActionChains(driver_k).send_keys(Keys.PAGE_DOWN).perform()  
        time.sleep(5)  # Wait for 5 seconds for the page to load more content

        new_page_source = driver_k.page_source  # Get the new page source

        # Stop scrolling if no new content is loaded
        if new_page_source == old_page_source:
            scrolling = False

    # Extract bus details such as name, type, departure time, etc.
    bus_name = driver_k.find_elements(By.XPATH, "//div[@class='travels lh-24 f-bold d-color']")
    bus_type = driver_k.find_elements(By.XPATH, "//div[@class='bus-type f-12 m-top-16 l-color evBus']")
    start_time = driver_k.find_elements(By.XPATH, "//*[@class='dp-time f-19 d-color f-bold']")
    end_time = driver_k.find_elements(By.XPATH, "//*[@class='bp-time f-19 d-color disp-Inline']")
    total_duration = driver_k.find_elements(By.XPATH, "//*[@class='dur l-color lh-24']")
    price = driver_k.find_elements(By.XPATH, '//div[@class="fare d-block"]//span')
    seats = driver_k.find_elements(By.XPATH, "//div[contains(@class, 'seat-left')]")

    # Attempt to extract ratings (if available)
    try:
        rating = driver_k.find_elements(By.XPATH, "//div[@class='clearfix row-one']/div[@class='column-six p-right-10 w-10 fl']")
    except:
        continue  # Skip to the next link if no ratings are found

    # Append extracted details to respective lists
    for bus in bus_name:
        Bus_names_k.append(bus.text)  # Append bus name to the list
        Route_links.append(link)  # Append route link to the list
        Route_names.append(routes)  # Append route name to the list
    for bus_type_elem in bus_type:
        Bus_types_k.append(bus_type_elem.text)  # Append bus type to the list
    for start_time_elem in start_time:
        Departure_k.append(start_time_elem.text)  # Append departure time to the list
    for end_time_elem in end_time:
        Arrival_k.append(end_time_elem.text)  # Append arrival time to the list
    for total_duration_elem in total_duration:
        Total_Duration_k.append(total_duration_elem.text)  # Append total duration to the list
    for ratings_elem in rating:
        Ratings_k.append(ratings_elem.text)  # Append ratings to the list
    for price_elem in price:
        Prices_k.append(price_elem.text)  # Append price to the list
    for seats_elem in seats:
        Seats_Available_k.append(seats_elem.text)  # Append available seats to the list

# Indicate successful completion
print("Successfully Completed")


Successfully Completed


In [20]:
# Create a dictionary where each key corresponds to a column name, and the value is the list of bus details
data = {
    'Bus_name': Bus_names_k,  # List containing the names of buses
    'Bus_type': Bus_types_k,  # List containing the types of buses (e.g., Sleeper, AC, etc.)
    'Start_time': Departure_k,  # List containing the departure times of the buses
    'End_time': Arrival_k,  # List containing the arrival times of the buses
    'Total_duration': Total_Duration_k,  # List containing the total travel duration for each bus
    'Price': Prices_k,  # List containing the price of each bus ticket
    "Seats_Available": Seats_Available_k,  # List containing the number of seats available for each bus
    "Ratings": Ratings_k,  # List containing the ratings for each bus
    'Route_name': Route_names,  # List containing the name of the routes (e.g., "Kozhikode to Ernakulam")
    'Route_link': Route_links  # List containing the URL links for each route
}

# Create a pandas DataFrame from the dictionary 'data' where each key-value pair becomes a column in the DataFrame
df_andra = pd.DataFrame(data)

# Define the path where the CSV file will be saved
path = r"/Users/priyankamalavade/Desktop/redbus_scraping/scraped_all_required_bus_details/df_andra.csv"

# Convert the DataFrame 'df_andra' into a CSV file and save it at the specified path
# index=False ensures that the row index is not included in the CSV file
df_andra.to_csv(path, index=False)

# Print a message indicating that the process has been completed successfully
print("Successfully Completed")


Successfully Completed


In [21]:
df_andra

,Bus_name,Bus_type,Start_time,End_time,Total_duration,Price,Seats_Available,Ratings,Route_name,Route_link
0,APSRTC - 3525,"SUPER LUXURY (NON-AC, 2 + 2 PUSH BACK)",15:00,23:00,08h 00m,480,31 Seats available,4.1\n15,Vijayawada to Hyderabad,https://www.redbus.in/bus-tickets/vijayawada-t...
1,APSRTC - 3886,DOLPHIN CRUISE (VOLVO / SCANIA A.C Multi Axle),15:00,20:00,05h 00m,670,26 Seats available,3.9\n73,Vijayawada to Hyderabad,https://www.redbus.in/bus-tickets/vijayawada-t...
2,APSRTC - 3573,AMARAVATHI (VOLVO / SCANIA A.C Multi Axle),15:30,21:25,05h 55m,720,17 Seats available,4.1\n96,Vijayawada to Hyderabad,https://www.redbus.in/bus-tickets/vijayawada-t...
3,APSRTC - 4052,"SUPER LUXURY (NON-AC, 2 + 2 PUSH BACK)",15:30,22:00,06h 30m,448,33 Seats available,3.4\n10,Vijayawada to Hyderabad,https://www.redbus.in/bus-tickets/vijayawada-t...
4,APSRTC - 35061,"SUPER LUXURY (NON-AC, 2 + 2 PUSH BACK)",15:45,00:10,08h 25m,509,33 Seats available,4.0,Vijayawada to Hyderabad,https://www.redbus.in/bus-tickets/vijayawada-t...
...,...,...,...,...,...,...,...,...,...,...
452,,,,,,,,,Hyderabad to Nandyal,https://www.redbus.in/bus-tickets/hyderabad-to...
453,,,,,,,,,Hyderabad to Nandyal,https://www.redbus.in/bus-tickets/hyderabad-to...
454,,,,,,,,,Hyderabad to Nandyal,https://www.redbus.in/bus-tickets/hyderabad-to...
455,,,,,,,,,Hyderabad to Nandyal,https://www.redbus.in/bus-tickets/hyderabad-to...


## ASSAM

In [4]:
# Load the CSV file into a DataFrame
df = pd.read_csv("scraped bus details/df.assam.csv")

# Display the DataFrame
df

,Route_name,Route_link
0,Tezpur to Guwahati,https://www.redbus.in/bus-tickets/tezpur-to-gu...
1,Guwahati to Tezpur,https://www.redbus.in/bus-tickets/guwahati-to-...
2,Nagaon (Assam) to Guwahati,https://www.redbus.in/bus-tickets/nagaon-to-gu...
3,Guwahati to Nagaon (Assam),https://www.redbus.in/bus-tickets/guwahati-to-...
4,Goalpara to Guwahati,https://www.redbus.in/bus-tickets/goalpara-to-...
5,Dhubri to Guwahati,https://www.redbus.in/bus-tickets/dhubri-to-gu...
6,Sibsagar (Assam) to North Lakhimpur,https://www.redbus.in/bus-tickets/sibsagar-to-...
7,North Lakhimpur to Sibsagar (Assam),https://www.redbus.in/bus-tickets/north-lakhim...
8,Guwahati to Dhubri,https://www.redbus.in/bus-tickets/guwahati-to-...
9,Jorhat to North Lakhimpur,https://www.redbus.in/bus-tickets/jorhat-to-no...


In [5]:

driver_k = webdriver.Chrome()  # Initialize the Chrome WebDriver
Bus_names_k = []  # List to store bus names
Bus_types_k = []  # List to store bus types
Departure_k = []  # List to store departure times
Arrival_k = []  # List to store arrival times
Ratings_k = []  # List to store ratings
Total_Duration_k = []  # List to store total travel duration
Prices_k = []  # List to store prices
Seats_Available_k = []  # List to store seat availability
Route_names = []  # List to store route names
Route_links = []  # List to store route links

# Loop through each row of the DataFrame df
for i, r in df.iterrows():  
    link = r["Route_link"]  # Get the route link from the DataFrame
    routes = r["Route_name"]  # Get the route name from the DataFrame

    # Open the route link in the browser
    driver_k.get(link)  
    time.sleep(2)  # Wait for 2 seconds for the page to load

    # Find clickable elements that contain the route link
    elements = driver_k.find_elements(By.XPATH, f"//a[contains(@href, '{link}')]")  
    for element in elements:
        element.click()  # Click the element to reveal bus details
        time.sleep(2)  # Wait for 2 seconds for the page to update

    # Try clicking the "View Buses" button if available
    try:
        clicks = driver_k.find_element(By.XPATH, "//div[@class='button']")
        clicks.click()  # Click the button to view buses
    except:
        continue  # Skip to the next link if the button is not found
    time.sleep(2)  # Wait for the page to update

    scrolling = True  # Initialize a flag to start scrolling
    while scrolling:
        old_page_source = driver_k.page_source  # Store the current page source

        # Scroll down the page using the PAGE_DOWN key
        ActionChains(driver_k).send_keys(Keys.PAGE_DOWN).perform()  
        time.sleep(5)  # Wait for 5 seconds for the page to load more content

        new_page_source = driver_k.page_source  # Get the new page source

        # Stop scrolling if no new content is loaded
        if new_page_source == old_page_source:
            scrolling = False

    # Extract bus details such as name, type, departure time, etc.
    bus_name = driver_k.find_elements(By.XPATH, "//div[@class='travels lh-24 f-bold d-color']")
    bus_type = driver_k.find_elements(By.XPATH, "//div[@class='bus-type f-12 m-top-16 l-color evBus']")
    start_time = driver_k.find_elements(By.XPATH, "//*[@class='dp-time f-19 d-color f-bold']")
    end_time = driver_k.find_elements(By.XPATH, "//*[@class='bp-time f-19 d-color disp-Inline']")
    total_duration = driver_k.find_elements(By.XPATH, "//*[@class='dur l-color lh-24']")
    price = driver_k.find_elements(By.XPATH, '//div[@class="fare d-block"]//span')
    seats = driver_k.find_elements(By.XPATH, "//div[contains(@class, 'seat-left')]")

    # Attempt to extract ratings (if available)
    try:
        rating = driver_k.find_elements(By.XPATH, "//div[@class='clearfix row-one']/div[@class='column-six p-right-10 w-10 fl']")
    except:
        continue  # Skip to the next link if no ratings are found

    # Append extracted details to respective lists
    for bus in bus_name:
        Bus_names_k.append(bus.text)  # Append bus name to the list
        Route_links.append(link)  # Append route link to the list
        Route_names.append(routes)  # Append route name to the list
    for bus_type_elem in bus_type:
        Bus_types_k.append(bus_type_elem.text)  # Append bus type to the list
    for start_time_elem in start_time:
        Departure_k.append(start_time_elem.text)  # Append departure time to the list
    for end_time_elem in end_time:
        Arrival_k.append(end_time_elem.text)  # Append arrival time to the list
    for total_duration_elem in total_duration:
        Total_Duration_k.append(total_duration_elem.text)  # Append total duration to the list
    for ratings_elem in rating:
        Ratings_k.append(ratings_elem.text)  # Append ratings to the list
    for price_elem in price:
        Prices_k.append(price_elem.text)  # Append price to the list
    for seats_elem in seats:
        Seats_Available_k.append(seats_elem.text)  # Append available seats to the list

# Indicate successful completion
print("Successfully Completed")

       

Successfully Completed


In [6]:
# Create a dictionary where each key corresponds to a column name, and the value is the list of bus details
data = {
    'Bus_name': Bus_names_k,  # List containing the names of buses
    'Bus_type': Bus_types_k,  # List containing the types of buses (e.g., Sleeper, AC, etc.)
    'Start_time': Departure_k,  # List containing the departure times of the buses
    'End_time': Arrival_k,  # List containing the arrival times of the buses
    'Total_duration': Total_Duration_k,  # List containing the total travel duration for each bus
    'Price': Prices_k,  # List containing the price of each bus ticket
    "Seats_Available": Seats_Available_k,  # List containing the number of seats available for each bus
    "Ratings": Ratings_k,  # List containing the ratings for each bus
    'Route_name': Route_names,  # List containing the name of the routes (e.g., "Kozhikode to Ernakulam")
    'Route_link': Route_links  # List containing the URL links for each route
}

# Create a pandas DataFrame from the dictionary 'data' where each key-value pair becomes a column in the DataFrame
df_assam = pd.DataFrame(data)

# Define the path where the CSV file will be saved
path = r"/Users/priyankamalavade/Desktop/redbus_scraping/scraped_all_required_bus_details/df_assam.csv"

# Convert the DataFrame 'df_assam' into a CSV file and save it at the specified path
# index=False ensures that the row index is not included in the CSV file
df_assam.to_csv(path, index=False)

# Print a message indicating that the process has been completed successfully
print("Successfully Completed")


Successfully Completed


In [7]:
df_assam

,Bus_name,Bus_type,Start_time,End_time,Total_duration,Price,Seats_Available,Ratings,Route_name,Route_link
0,Assam State Transport Corporation (ASTC) - 147480,Bharat Benz A/C Seater (2+2),05:00,09:00,04h 00m,270,11 Seats available,3.6\n344,Tezpur to Guwahati,https://www.redbus.in/bus-tickets/tezpur-to-gu...
1,Assam State Transport Corporation (ASTC) - 135927,Volvo AC Seater 2+2,07:10,11:40,04h 30m,298,20 Seats available,4.2\n317,Tezpur to Guwahati,https://www.redbus.in/bus-tickets/tezpur-to-gu...
2,Assam State Transport Corporation (ASTC) - 173420,Bharat Benz A/C Seater (2+2),10:35,15:15,04h 40m,298,2 Seats available,4.2\n77,Tezpur to Guwahati,https://www.redbus.in/bus-tickets/tezpur-to-gu...
3,Assam State Transport Corporation (ASTC) - 176210,Tata A/C Seater (2+2),16:15,20:30,04h 15m,298,25 Seats available,4.0,Tezpur to Guwahati,https://www.redbus.in/bus-tickets/tezpur-to-gu...
4,NEELKANTH TRAVELS,A/C Seater Push Back (2+2),05:15,08:30,03h 15m,300,30 Seats available,4.3\n189,Tezpur to Guwahati,https://www.redbus.in/bus-tickets/tezpur-to-gu...
5,Kanchan Travels,AC Seater (2+2),05:20,08:30,03h 10m,277,34 Seats available,3.9\n53,Tezpur to Guwahati,https://www.redbus.in/bus-tickets/tezpur-to-gu...
6,Mahi Travels(Under ASTC),A/C Seater (2+2),05:40,08:55,03h 15m,357,24 Seats available,4.6\n350,Tezpur to Guwahati,https://www.redbus.in/bus-tickets/tezpur-to-gu...
7,WARISPIYA TRAVELS,AC Seater (2+2),05:45,09:00,03h 15m,360,29 Seats available,4.3\n331,Tezpur to Guwahati,https://www.redbus.in/bus-tickets/tezpur-to-gu...
8,Orient Transline,A/C Seater (2+1),05:55,09:00,03h 05m,361.9,26 Seats available,4.5\n86,Tezpur to Guwahati,https://www.redbus.in/bus-tickets/tezpur-to-gu...
9,Puja (Kaybom),A/C Seater Push Back (2+2),06:10,10:00,03h 50m,314,40 Seats available,3.3\n15,Tezpur to Guwahati,https://www.redbus.in/bus-tickets/tezpur-to-gu...


## BIHAR

In [9]:
# Load the CSV file into a DataFrame
df = pd.read_csv("scraped bus details/df.bihar.csv")

# Display the DataFrame
df

,Route_name,Route_link
0,Patna (Bihar) to Bettiah,https://www.redbus.in/bus-tickets/patna-to-bet...
1,Patna (Bihar) to Motihari,https://www.redbus.in/bus-tickets/patna-to-mot...
2,Bettiah to Patna (Bihar),https://www.redbus.in/bus-tickets/bettiah-to-p...
3,Gopalganj (Bihar) to Delhi,https://www.redbus.in/bus-tickets/gopalganj-to...
4,Patna (Bihar) to Balmiki Nagar (bihar),https://www.redbus.in/bus-tickets/patna-to-bal...
5,Balmiki Nagar (bihar) to Patna (Bihar),https://www.redbus.in/bus-tickets/balmiki-naga...
6,Delhi to Motihari,https://www.redbus.in/bus-tickets/delhi-to-mot...
7,Patna (Bihar) to Katihar,https://www.redbus.in/bus-tickets/patna-to-kat...
8,Patna (Bihar) to Purnea,https://www.redbus.in/bus-tickets/patna-to-purnea
9,Motihari to Delhi,https://www.redbus.in/bus-tickets/motihari-to-...


In [10]:
driver_k = webdriver.Chrome()  # Initialize the Chrome WebDriver
Bus_names_k = []  # List to store bus names
Bus_types_k = []  # List to store bus types
Departure_k = []  # List to store departure times
Arrival_k = []  # List to store arrival times
Ratings_k = []  # List to store ratings
Total_Duration_k = []  # List to store total travel duration
Prices_k = []  # List to store prices
Seats_Available_k = []  # List to store seat availability
Route_names = []  # List to store route names
Route_links = []  # List to store route links

# Loop through each row of the DataFrame df
for i, r in df.iterrows():  
    link = r["Route_link"]  # Get the route link from the DataFrame
    routes = r["Route_name"]  # Get the route name from the DataFrame

    # Open the route link in the browser
    driver_k.get(link)  
    time.sleep(2)  # Wait for 2 seconds for the page to load

    # Find clickable elements that contain the route link
    elements = driver_k.find_elements(By.XPATH, f"//a[contains(@href, '{link}')]")  
    for element in elements:
        element.click()  # Click the element to reveal bus details
        time.sleep(2)  # Wait for 2 seconds for the page to update

    # Try clicking the "View Buses" button if available
    try:
        clicks = driver_k.find_element(By.XPATH, "//div[@class='button']")
        clicks.click()  # Click the button to view buses
    except:
        continue  # Skip to the next link if the button is not found
    time.sleep(2)  # Wait for the page to update

    scrolling = True  # Initialize a flag to start scrolling
    while scrolling:
        old_page_source = driver_k.page_source  # Store the current page source

        # Scroll down the page using the PAGE_DOWN key
        ActionChains(driver_k).send_keys(Keys.PAGE_DOWN).perform()  
        time.sleep(5)  # Wait for 5 seconds for the page to load more content

        new_page_source = driver_k.page_source  # Get the new page source

        # Stop scrolling if no new content is loaded
        if new_page_source == old_page_source:
            scrolling = False

    # Extract bus details such as name, type, departure time, etc.
    bus_name = driver_k.find_elements(By.XPATH, "//div[@class='travels lh-24 f-bold d-color']")
    bus_type = driver_k.find_elements(By.XPATH, "//div[@class='bus-type f-12 m-top-16 l-color evBus']")
    start_time = driver_k.find_elements(By.XPATH, "//*[@class='dp-time f-19 d-color f-bold']")
    end_time = driver_k.find_elements(By.XPATH, "//*[@class='bp-time f-19 d-color disp-Inline']")
    total_duration = driver_k.find_elements(By.XPATH, "//*[@class='dur l-color lh-24']")
    price = driver_k.find_elements(By.XPATH, '//div[@class="fare d-block"]//span')
    seats = driver_k.find_elements(By.XPATH, "//div[contains(@class, 'seat-left')]")

    # Attempt to extract ratings (if available)
    try:
        rating = driver_k.find_elements(By.XPATH, "//div[@class='clearfix row-one']/div[@class='column-six p-right-10 w-10 fl']")
    except:
        continue  # Skip to the next link if no ratings are found

    # Append extracted details to respective lists
    for bus in bus_name:
        Bus_names_k.append(bus.text)  # Append bus name to the list
        Route_links.append(link)  # Append route link to the list
        Route_names.append(routes)  # Append route name to the list
    for bus_type_elem in bus_type:
        Bus_types_k.append(bus_type_elem.text)  # Append bus type to the list
    for start_time_elem in start_time:
        Departure_k.append(start_time_elem.text)  # Append departure time to the list
    for end_time_elem in end_time:
        Arrival_k.append(end_time_elem.text)  # Append arrival time to the list
    for total_duration_elem in total_duration:
        Total_Duration_k.append(total_duration_elem.text)  # Append total duration to the list
    for ratings_elem in rating:
        Ratings_k.append(ratings_elem.text)  # Append ratings to the list
    for price_elem in price:
        Prices_k.append(price_elem.text)  # Append price to the list
    for seats_elem in seats:
        Seats_Available_k.append(seats_elem.text)  # Append available seats to the list

# Indicate successful completion
print("Successfully Completed")


Successfully Completed


In [11]:
# Create a dictionary where each key corresponds to a column name, and the value is the list of bus details
data = {
    'Bus_name': Bus_names_k,  # List containing the names of buses
    'Bus_type': Bus_types_k,  # List containing the types of buses (e.g., Sleeper, AC, etc.)
    'Start_time': Departure_k,  # List containing the departure times of the buses
    'End_time': Arrival_k,  # List containing the arrival times of the buses
    'Total_duration': Total_Duration_k,  # List containing the total travel duration for each bus
    'Price': Prices_k,  # List containing the price of each bus ticket
    "Seats_Available": Seats_Available_k,  # List containing the number of seats available for each bus
    "Ratings": Ratings_k,  # List containing the ratings for each bus
    'Route_name': Route_names,  # List containing the name of the routes (e.g., "Kozhikode to Ernakulam")
    'Route_link': Route_links  # List containing the URL links for each route
}

# Create a pandas DataFrame from the dictionary 'data' where each key-value pair becomes a column in the DataFrame
df_bihar = pd.DataFrame(data)

# Define the path where the CSV file will be saved
path = r"/Users/priyankamalavade/Desktop/redbus_scraping/scraped_all_required_bus_details/df_bihar.csv"

# Convert the DataFrame 'df_bihar' into a CSV file and save it at the specified path
# index=False ensures that the row index is not included in the CSV file
df_bihar.to_csv(path, index=False)

# Print a message indicating that the process has been completed successfully
print("Successfully Completed")


Successfully Completed


In [12]:
df_bihar

,Bus_name,Bus_type,Start_time,End_time,Total_duration,Price,Seats_Available,Ratings,Route_name,Route_link
0,Bihar state road transport corporation (BSRTC)...,Non AC Seater (2+2),06:00,11:00,05h 00m,285,31 Seats available,3.6\n137,Patna (Bihar) to Bettiah,https://www.redbus.in/bus-tickets/patna-to-bet...
1,Bihar state road transport corporation (BSRTC)...,AC Seater (2+2),06:30,12:35,06h 05m,331,23 Seats available,3.3\n159,Patna (Bihar) to Bettiah,https://www.redbus.in/bus-tickets/patna-to-bet...
2,Bihar state road transport corporation (BSRTC)...,Non AC Seater (2+2),09:00,15:00,06h 00m,285,31 Seats available,3.3\n30,Patna (Bihar) to Bettiah,https://www.redbus.in/bus-tickets/patna-to-bet...
3,Jai Mata Di,AC Seater (2+2),05:40,12:05,06h 25m,450,41 Seats available,3.7,Patna (Bihar) to Bettiah,https://www.redbus.in/bus-tickets/patna-to-bet...
4,Jai Mata Di,AC Seater (2+2),05:43,12:00,06h 17m,450,41 Seats available,3.7\n2,Patna (Bihar) to Bettiah,https://www.redbus.in/bus-tickets/patna-to-bet...
5,Jai Mata Di,AC Seater (2+2),05:45,11:50,06h 05m,450,41 Seats available,3.7,Patna (Bihar) to Bettiah,https://www.redbus.in/bus-tickets/patna-to-bet...
6,Jai Mata Di,AC Seater (2+2),05:47,12:03,06h 16m,450,41 Seats available,3.7,Patna (Bihar) to Bettiah,https://www.redbus.in/bus-tickets/patna-to-bet...
7,Jai Hanuman Travels,A/C Seater / Sleeper (2+2),22:20,05:00,06h 40m,550,54 Seats available,3.6\n47,Patna (Bihar) to Bettiah,https://www.redbus.in/bus-tickets/patna-to-bet...
8,Jai Mata Di Tour and Travels,A/C Seater / Sleeper (2+2),22:40,04:00,05h 20m,405,28 Seats available,3.4\n21,Patna (Bihar) to Bettiah,https://www.redbus.in/bus-tickets/patna-to-bet...
9,Saurabh Travels,A/C Seater Push Back (2+2),07:00,12:30,05h 30m,450,42 Seats available,1.0,Patna (Bihar) to Bettiah,https://www.redbus.in/bus-tickets/patna-to-bet...


## CHANDIGARH

In [13]:
# Load the CSV file into a DataFrame
df = pd.read_csv("scraped bus details/df.bihar.csv")

# Display the DataFrame
df

,Route_name,Route_link
0,Delhi to Chandigarh,https://www.redbus.in/bus-tickets/delhi-to-cha...
1,Chandigarh to Delhi,https://www.redbus.in/bus-tickets/chandigarh-t...
2,Yamuna Nagar to Chandigarh,https://www.redbus.in/bus-tickets/yamuna-nagar...
3,Ludhiana to Chandigarh,https://www.redbus.in/bus-tickets/ludhiana-to-...
4,Chandigarh to Yamuna Nagar,https://www.redbus.in/bus-tickets/chandigarh-t...
5,Chandigarh to Hamirpur (Himachal Pradesh),https://www.redbus.in/bus-tickets/chandigarh-t...
6,Chandigarh to Baijnath,https://www.redbus.in/bus-tickets/chandigarh-t...
7,Hamirpur (Himachal Pradesh) to Chandigarh,https://www.redbus.in/bus-tickets/hamirpur-him...
8,Chandigarh to Ludhiana,https://www.redbus.in/bus-tickets/chandigarh-t...
9,Chandigarh to Dehradun,https://www.redbus.in/bus-tickets/chandigarh-t...


In [14]:
driver_k = webdriver.Chrome()  # Initialize the Chrome WebDriver
Bus_names_k = []  # List to store bus names
Bus_types_k = []  # List to store bus types
Departure_k = []  # List to store departure times
Arrival_k = []  # List to store arrival times
Ratings_k = []  # List to store ratings
Total_Duration_k = []  # List to store total travel duration
Prices_k = []  # List to store prices
Seats_Available_k = []  # List to store seat availability
Route_names = []  # List to store route names
Route_links = []  # List to store route links

# Loop through each row of the DataFrame df
for i, r in df.iterrows():  
    link = r["Route_link"]  # Get the route link from the DataFrame
    routes = r["Route_name"]  # Get the route name from the DataFrame

    # Open the route link in the browser
    driver_k.get(link)  
    time.sleep(2)  # Wait for 2 seconds for the page to load

    # Find clickable elements that contain the route link
    elements = driver_k.find_elements(By.XPATH, f"//a[contains(@href, '{link}')]")  
    for element in elements:
        element.click()  # Click the element to reveal bus details
        time.sleep(2)  # Wait for 2 seconds for the page to update

    # Try clicking the "View Buses" button if available
    try:
        clicks = driver_k.find_element(By.XPATH, "//div[@class='button']")
        clicks.click()  # Click the button to view buses
    except:
        continue  # Skip to the next link if the button is not found
    time.sleep(2)  # Wait for the page to update

    scrolling = True  # Initialize a flag to start scrolling
    while scrolling:
        old_page_source = driver_k.page_source  # Store the current page source

        # Scroll down the page using the PAGE_DOWN key
        ActionChains(driver_k).send_keys(Keys.PAGE_DOWN).perform()  
        time.sleep(5)  # Wait for 5 seconds for the page to load more content

        new_page_source = driver_k.page_source  # Get the new page source

        # Stop scrolling if no new content is loaded
        if new_page_source == old_page_source:
            scrolling = False

    # Extract bus details such as name, type, departure time, etc.
    bus_name = driver_k.find_elements(By.XPATH, "//div[@class='travels lh-24 f-bold d-color']")
    bus_type = driver_k.find_elements(By.XPATH, "//div[@class='bus-type f-12 m-top-16 l-color evBus']")
    start_time = driver_k.find_elements(By.XPATH, "//*[@class='dp-time f-19 d-color f-bold']")
    end_time = driver_k.find_elements(By.XPATH, "//*[@class='bp-time f-19 d-color disp-Inline']")
    total_duration = driver_k.find_elements(By.XPATH, "//*[@class='dur l-color lh-24']")
    price = driver_k.find_elements(By.XPATH, '//div[@class="fare d-block"]//span')
    seats = driver_k.find_elements(By.XPATH, "//div[contains(@class, 'seat-left')]")

    # Attempt to extract ratings (if available)
    try:
        rating = driver_k.find_elements(By.XPATH, "//div[@class='clearfix row-one']/div[@class='column-six p-right-10 w-10 fl']")
    except:
        continue  # Skip to the next link if no ratings are found

    # Append extracted details to respective lists
    for bus in bus_name:
        Bus_names_k.append(bus.text)  # Append bus name to the list
        Route_links.append(link)  # Append route link to the list
        Route_names.append(routes)  # Append route name to the list
    for bus_type_elem in bus_type:
        Bus_types_k.append(bus_type_elem.text)  # Append bus type to the list
    for start_time_elem in start_time:
        Departure_k.append(start_time_elem.text)  # Append departure time to the list
    for end_time_elem in end_time:
        Arrival_k.append(end_time_elem.text)  # Append arrival time to the list
    for total_duration_elem in total_duration:
        Total_Duration_k.append(total_duration_elem.text)  # Append total duration to the list
    for ratings_elem in rating:
        Ratings_k.append(ratings_elem.text)  # Append ratings to the list
    for price_elem in price:
        Prices_k.append(price_elem.text)  # Append price to the list
    for seats_elem in seats:
        Seats_Available_k.append(seats_elem.text)  # Append available seats to the list

# Indicate successful completion
print("Successfully Completed")


Successfully Completed


In [15]:
# Create a dictionary where each key corresponds to a column name, and the value is the list of bus details
data = {
    'Bus_name': Bus_names_k,  # List containing the names of buses
    'Bus_type': Bus_types_k,  # List containing the types of buses (e.g., Sleeper, AC, etc.)
    'Start_time': Departure_k,  # List containing the departure times of the buses
    'End_time': Arrival_k,  # List containing the arrival times of the buses
    'Total_duration': Total_Duration_k,  # List containing the total travel duration for each bus
    'Price': Prices_k,  # List containing the price of each bus ticket
    "Seats_Available": Seats_Available_k,  # List containing the number of seats available for each bus
    "Ratings": Ratings_k,  # List containing the ratings for each bus
    'Route_name': Route_names,  # List containing the name of the routes (e.g., "Kozhikode to Ernakulam")
    'Route_link': Route_links  # List containing the URL links for each route
}

# Create a pandas DataFrame from the dictionary 'data' where each key-value pair becomes a column in the DataFrame
df_chandigarh = pd.DataFrame(data)

# Define the path where the CSV file will be saved
path = r"/Users/priyankamalavade/Desktop/redbus_scraping/scraped_all_required_bus_details/df_chandigarh.csv"

# Convert the DataFrame 'df_chandigarh' into a CSV file and save it at the specified path
# index=False ensures that the row index is not included in the CSV file
df_chandigarh.to_csv(path, index=False)

# Print a message indicating that the process has been completed successfully
print("Successfully Completed")


Successfully Completed


In [16]:
df_chandigarh

,Bus_name,Bus_type,Start_time,End_time,Total_duration,Price,Seats_Available,Ratings,Route_name,Route_link
0,Chandigarh Transport Undertaking (CTU) - 165681,HVAC Seater (2+3),07:00,12:35,05h 35m,414.3,45 Seats available,4.3\n67,Chandigarh to Delhi,https://www.redbus.in/bus-tickets/chandigarh-t...
1,Chandigarh Transport Undertaking (CTU) - 165684,HVAC Seater (2+3),07:55,13:30,05h 35m,414.3,45 Seats available,3.8\n25,Chandigarh to Delhi,https://www.redbus.in/bus-tickets/chandigarh-t...
2,Chandigarh Transport Undertaking (CTU) - 165687,HVAC Seater (2+3),08:25,14:00,05h 35m,414.3,47 Seats available,4.2\n36,Chandigarh to Delhi,https://www.redbus.in/bus-tickets/chandigarh-t...
3,Chandigarh Transport Undertaking (CTU) - 165676,HVAC Seater (2+3),09:25,15:00,05h 35m,414.3,45 Seats available,4.4\n39,Chandigarh to Delhi,https://www.redbus.in/bus-tickets/chandigarh-t...
4,Chandigarh Transport Undertaking (CTU) - 165689,HVAC Seater (2+3),10:30,16:05,05h 35m,414.3,47 Seats available,4.0\n44,Chandigarh to Delhi,https://www.redbus.in/bus-tickets/chandigarh-t...
...,...,...,...,...,...,...,...,...,...,...
164,RSRTC - 192348,Super Luxury Volvo AC Seater Pushback 2+2,16:30,21:15,04h 45m,432,21 Seats available,4.4\n20,Shimla to Chandigarh,https://www.redbus.in/bus-tickets/shimla-to-ch...
165,FlixBus,Volvo A/C Seater (2+2),10:30,14:55,04h 25m,634.82,4 Seats available,3.7\n50,Shimla to Chandigarh,https://www.redbus.in/bus-tickets/shimla-to-ch...
166,FlixBus,Volvo A/C Seater (2+2),21:00,01:25,04h 25m,616.07,10 Seats available,3.4\n17,Shimla to Chandigarh,https://www.redbus.in/bus-tickets/shimla-to-ch...
167,,,,,,,,,Shimla to Chandigarh,https://www.redbus.in/bus-tickets/shimla-to-ch...


## GOA

In [3]:
# Load the CSV file into a DataFrame
df = pd.read_csv("scraped bus details/df.GOA.csv")

# Display the DataFrame
df

,Route_name,Route_link
0,Pune to Goa,https://www.redbus.in/bus-tickets/pune-to-goa
1,Goa to Pune,https://www.redbus.in/bus-tickets/goa-to-pune
2,Mumbai to Goa,https://www.redbus.in/bus-tickets/mumbai-to-goa
3,Goa to Mumbai,https://www.redbus.in/bus-tickets/goa-to-mumbai
4,Belagavi to Goa,https://www.redbus.in/bus-tickets/belagavi-to-goa
5,Bangalore to Goa,https://www.redbus.in/bus-tickets/bangalore-to...
6,Goa to Bangalore,https://www.redbus.in/bus-tickets/goa-to-banga...
7,Pandharpur to Goa,https://www.redbus.in/bus-tickets/pandharpur-t...
8,Goa to Belagavi,https://www.redbus.in/bus-tickets/goa-to-belagavi
9,Goa to Kolhapur(Maharashtra),https://www.redbus.in/bus-tickets/goa-to-kolha...


In [11]:
driver_k = webdriver.Chrome()  # Initialize the Chrome WebDriver
Bus_names_k = []  # List to store bus names
Bus_types_k = []  # List to store bus types
Departure_k = []  # List to store departure times
Arrival_k = []  # List to store arrival times
Ratings_k = []  # List to store ratings
Total_Duration_k = []  # List to store total travel duration
Prices_k = []  # List to store prices
Seats_Available_k = []  # List to store seat availability
Route_names = []  # List to store route names
Route_links = []  # List to store route links

# Loop through each row of the DataFrame df
for i, r in df.iterrows():  
    link = r["Route_link"]  # Get the route link from the DataFrame
    routes = r["Route_name"]  # Get the route name from the DataFrame

    # Open the route link in the browser
    driver_k.get(link)  
    time.sleep(2)  # Wait for 2 seconds for the page to load

    # Find clickable elements that contain the route link
    elements = driver_k.find_elements(By.XPATH, f"//a[contains(@href, '{link}')]")  
    for element in elements:
        element.click()  # Click the element to reveal bus details
        time.sleep(2)  # Wait for 2 seconds for the page to update

    # Try clicking the "View Buses" button if available
    try:
        clicks = driver_k.find_element(By.XPATH, "//div[@class='button']")
        clicks.click()  # Click the button to view buses
    except:
        continue  # Skip to the next link if the button is not found
    time.sleep(2)  # Wait for the page to update

    scrolling = True  # Initialize a flag to start scrolling
    while scrolling:
        old_page_source = driver_k.page_source  # Store the current page source

        # Scroll down the page using the PAGE_DOWN key
        ActionChains(driver_k).send_keys(Keys.PAGE_DOWN).perform()  
        time.sleep(5)  # Wait for 5 seconds for the page to load more content

        new_page_source = driver_k.page_source  # Get the new page source

        # Stop scrolling if no new content is loaded
        if new_page_source == old_page_source:
            scrolling = False

    # Extract bus details such as name, type, departure time, etc.
    bus_name = driver_k.find_elements(By.XPATH, "//div[@class='travels lh-24 f-bold d-color']")
    bus_type = driver_k.find_elements(By.XPATH, "//div[@class='bus-type f-12 m-top-16 l-color evBus']")
    start_time = driver_k.find_elements(By.XPATH, "//*[@class='dp-time f-19 d-color f-bold']")
    end_time = driver_k.find_elements(By.XPATH, "//*[@class='bp-time f-19 d-color disp-Inline']")
    total_duration = driver_k.find_elements(By.XPATH, "//*[@class='dur l-color lh-24']")
    price = driver_k.find_elements(By.XPATH, '//div[@class="fare d-block"]//span')
    seats = driver_k.find_elements(By.XPATH, "//div[contains(@class, 'seat-left')]")

    # Attempt to extract ratings (if available)
    try:
        rating = driver_k.find_elements(By.XPATH, "//div[@class='clearfix row-one']/div[@class='column-six p-right-10 w-10 fl']")
    except:
        continue  # Skip to the next link if no ratings are found

    # Append extracted details to respective lists
    for bus in bus_name:
        Bus_names_k.append(bus.text)  # Append bus name to the list
        Route_links.append(link)  # Append route link to the list
        Route_names.append(routes)  # Append route name to the list
    for bus_type_elem in bus_type:
        Bus_types_k.append(bus_type_elem.text)  # Append bus type to the list
    for start_time_elem in start_time:
        Departure_k.append(start_time_elem.text)  # Append departure time to the list
    for end_time_elem in end_time:
        Arrival_k.append(end_time_elem.text)  # Append arrival time to the list
    for total_duration_elem in total_duration:
        Total_Duration_k.append(total_duration_elem.text)  # Append total duration to the list
    for ratings_elem in rating:
        Ratings_k.append(ratings_elem.text)  # Append ratings to the list
    for price_elem in price:
        Prices_k.append(price_elem.text)  # Append price to the list
    for seats_elem in seats:
        Seats_Available_k.append(seats_elem.text)  # Append available seats to the list

# Indicate successful completion
print("Successfully Completed")

 

Successfully Completed


In [12]:
# Create a dictionary where each key corresponds to a column name, and the value is the list of bus details
data = {
    'Bus_name': Bus_names_k,  # List containing the names of buses
    'Bus_type': Bus_types_k,  # List containing the types of buses (e.g., Sleeper, AC, etc.)
    'Start_time': Departure_k,  # List containing the departure times of the buses
    'End_time': Arrival_k,  # List containing the arrival times of the buses
    'Total_duration': Total_Duration_k,  # List containing the total travel duration for each bus
    'Price': Prices_k,  # List containing the price of each bus ticket
    "Seats_Available": Seats_Available_k,  # List containing the number of seats available for each bus
    "Ratings": Ratings_k,  # List containing the ratings for each bus
    'Route_name': Route_names,  # List containing the name of the routes (e.g., "Kozhikode to Ernakulam")
    'Route_link': Route_links  # List containing the URL links for each route
}

# Create a pandas DataFrame from the dictionary 'data' where each key-value pair becomes a column in the DataFrame
df_GOA = pd.DataFrame(data)

# Define the path where the CSV file will be saved
path = r"/Users/priyankamalavade/Desktop/redbus_scraping/scraped_all_required_bus_details/df_GOA.csv"

# Convert the DataFrame 'df_GOA' into a CSV file and save it at the specified path
# index=False ensures that the row index is not included in the CSV file
df_GOA.to_csv(path, index=False)

# Print a message indicating that the process has been completed successfully
print("Successfully Completed")


Successfully Completed


In [14]:
df_GOA

,Bus_name,Bus_type,Start_time,End_time,Total_duration,Price,Seats_Available,Ratings,Route_name,Route_link


## HIMACHAL PRADESH

In [24]:
# Load the CSV file into a DataFrame
df = pd.read_csv("scraped bus details/df.himachalpradesh.csv")

# Display the DataFrame
df

,Route_name,Route_link
0,Delhi to Shimla,https://www.redbus.in/bus-tickets/delhi-to-shimla
1,Hamirpur (Himachal Pradesh) to Chandigarh,https://www.redbus.in/bus-tickets/hamirpur-him...
2,Chandigarh to Hamirpur (Himachal Pradesh),https://www.redbus.in/bus-tickets/chandigarh-t...
3,Shimla to Delhi,https://www.redbus.in/bus-tickets/shimla-to-delhi
4,Hamirpur (Himachal Pradesh) to Delhi,https://www.redbus.in/bus-tickets/hamirpur-him...
5,Delhi to Hamirpur (Himachal Pradesh),https://www.redbus.in/bus-tickets/delhi-to-ham...
6,Kangra to Chandigarh,https://www.redbus.in/bus-tickets/kangra-to-ch...
7,Chamba (Himachal Pradesh) to Chandigarh,https://www.redbus.in/bus-tickets/chamba-himac...
8,Delhi to Chandigarh,https://www.redbus.in/bus-tickets/delhi-to-cha...
9,Palampur to Chandigarh,https://www.redbus.in/bus-tickets/palampur-to-...


In [25]:
driver_k = webdriver.Chrome()  # Initialize the Chrome WebDriver
Bus_names_k = []  # List to store bus names
Bus_types_k = []  # List to store bus types
Departure_k = []  # List to store departure times
Arrival_k = []  # List to store arrival times
Ratings_k = []  # List to store ratings
Total_Duration_k = []  # List to store total travel duration
Prices_k = []  # List to store prices
Seats_Available_k = []  # List to store seat availability
Route_names = []  # List to store route names
Route_links = []  # List to store route links

# Loop through each row of the DataFrame df
for i, r in df.iterrows():  
    link = r["Route_link"]  # Get the route link from the DataFrame
    routes = r["Route_name"]  # Get the route name from the DataFrame

    # Open the route link in the browser
    driver_k.get(link)  
    time.sleep(2)  # Wait for 2 seconds for the page to load

    # Find clickable elements that contain the route link
    elements = driver_k.find_elements(By.XPATH, f"//a[contains(@href, '{link}')]")  
    for element in elements:
        element.click()  # Click the element to reveal bus details
        time.sleep(2)  # Wait for 2 seconds for the page to update

    # Try clicking the "View Buses" button if available
    try:
        clicks = driver_k.find_element(By.XPATH, "//div[@class='button']")
        clicks.click()  # Click the button to view buses
    except:
        continue  # Skip to the next link if the button is not found
    time.sleep(2)  # Wait for the page to update

    scrolling = True  # Initialize a flag to start scrolling
    while scrolling:
        old_page_source = driver_k.page_source  # Store the current page source

        # Scroll down the page using the PAGE_DOWN key
        ActionChains(driver_k).send_keys(Keys.PAGE_DOWN).perform()  
        time.sleep(5)  # Wait for 5 seconds for the page to load more content

        new_page_source = driver_k.page_source  # Get the new page source

        # Stop scrolling if no new content is loaded
        if new_page_source == old_page_source:
            scrolling = False

    # Extract bus details such as name, type, departure time, etc.
    bus_name = driver_k.find_elements(By.XPATH, "//div[@class='travels lh-24 f-bold d-color']")
    bus_type = driver_k.find_elements(By.XPATH, "//div[@class='bus-type f-12 m-top-16 l-color evBus']")
    start_time = driver_k.find_elements(By.XPATH, "//*[@class='dp-time f-19 d-color f-bold']")
    end_time = driver_k.find_elements(By.XPATH, "//*[@class='bp-time f-19 d-color disp-Inline']")
    total_duration = driver_k.find_elements(By.XPATH, "//*[@class='dur l-color lh-24']")
    price = driver_k.find_elements(By.XPATH, '//div[@class="fare d-block"]//span')
    seats = driver_k.find_elements(By.XPATH, "//div[contains(@class, 'seat-left')]")

    # Attempt to extract ratings (if available)
    try:
        rating = driver_k.find_elements(By.XPATH, "//div[@class='clearfix row-one']/div[@class='column-six p-right-10 w-10 fl']")
    except:
        continue  # Skip to the next link if no ratings are found

    # Append extracted details to respective lists
    for bus in bus_name:
        Bus_names_k.append(bus.text)  # Append bus name to the list
        Route_links.append(link)  # Append route link to the list
        Route_names.append(routes)  # Append route name to the list
    for bus_type_elem in bus_type:
        Bus_types_k.append(bus_type_elem.text)  # Append bus type to the list
    for start_time_elem in start_time:
        Departure_k.append(start_time_elem.text)  # Append departure time to the list
    for end_time_elem in end_time:
        Arrival_k.append(end_time_elem.text)  # Append arrival time to the list
    for total_duration_elem in total_duration:
        Total_Duration_k.append(total_duration_elem.text)  # Append total duration to the list
    for ratings_elem in rating:
        Ratings_k.append(ratings_elem.text)  # Append ratings to the list
    for price_elem in price:
        Prices_k.append(price_elem.text)  # Append price to the list
    for seats_elem in seats:
        Seats_Available_k.append(seats_elem.text)  # Append available seats to the list

# Indicate successful completion
print("Successfully Completed")


Successfully Completed


In [26]:
# Create a dictionary where each key corresponds to a column name, and the value is the list of bus details
data = {
    'Bus_name': Bus_names_k,  # List containing the names of buses
    'Bus_type': Bus_types_k,  # List containing the types of buses (e.g., Sleeper, AC, etc.)
    'Start_time': Departure_k,  # List containing the departure times of the buses
    'End_time': Arrival_k,  # List containing the arrival times of the buses
    'Total_duration': Total_Duration_k,  # List containing the total travel duration for each bus
    'Price': Prices_k,  # List containing the price of each bus ticket
    "Seats_Available": Seats_Available_k,  # List containing the number of seats available for each bus
    "Ratings": Ratings_k,  # List containing the ratings for each bus
    'Route_name': Route_names,  # List containing the name of the routes (e.g., "Kozhikode to Ernakulam")
    'Route_link': Route_links  # List containing the URL links for each route
}

# Create a pandas DataFrame from the dictionary 'data' where each key-value pair becomes a column in the DataFrame
df_himachalpradesh = pd.DataFrame(data)

# Define the path where the CSV file will be saved
path = r"/Users/priyankamalavade/Desktop/redbus_scraping/scraped_all_required_bus_details/df_himachalpradesh.csv"

# Convert the DataFrame 'df_himachalpradesh' into a CSV file and save it at the specified path
# index=False ensures that the row index is not included in the CSV file
df_himachalpradesh.to_csv(path, index=False)

# Print a message indicating that the process has been completed successfully
print("Successfully Completed")

Successfully Completed


In [27]:
df_himachalpradesh

,Bus_name,Bus_type,Start_time,End_time,Total_duration,Price,Seats_Available,Ratings,Route_name,Route_link
0,HRTC - 69,Himsuta AC Seater Volvo/Scania 2+2,00:40,09:30,08h 50m,912,25 Seats available,4.6\n61,Delhi to Shimla,https://www.redbus.in/bus-tickets/delhi-to-shimla
1,HRTC - 2100720,Ordinary 3+2 Non AC Seater,05:00,14:50,09h 50m,512,35 Seats available,4.1,Delhi to Shimla,https://www.redbus.in/bus-tickets/delhi-to-shimla
2,HRTC - 6,Himsuta AC Seater Volvo/Scania 2+2,06:45,16:10,09h 25m,912,24 Seats available,4.7\n55,Delhi to Shimla,https://www.redbus.in/bus-tickets/delhi-to-shimla
3,HRTC - 592,A/C Executive (2+3),08:05,18:10,10h 05m,632,39 Seats available,3.7\n15,Delhi to Shimla,https://www.redbus.in/bus-tickets/delhi-to-shimla
4,HRTC - 129,Ordinary 3+2 Non AC Seater,08:50,18:40,09h 50m,512,37 Seats available,3.8\n5,Delhi to Shimla,https://www.redbus.in/bus-tickets/delhi-to-shimla
...,...,...,...,...,...,...,...,...,...,...
233,HRTC - 812,Ordinary 3+2 Non AC Seater,09:20,19:30,10h 10m,556,34 Seats available,4.2\n6,Manali to Shimla,https://www.redbus.in/bus-tickets/manali-to-sh...
234,HRTC - 710,Himmani Deluxe 2+2 Non AC Seater,18:20,04:30,10h 10m,641,23 Seats available,3.7\n17,Manali to Shimla,https://www.redbus.in/bus-tickets/manali-to-sh...
235,HRTC - 726,Ordinary 3+2 Non AC Seater,19:20,04:30,09h 10m,583,27 Seats available,3.7\n4,Manali to Shimla,https://www.redbus.in/bus-tickets/manali-to-sh...
236,,,,,,,,,Manali to Shimla,https://www.redbus.in/bus-tickets/manali-to-sh...


## JAMMU KASHMIR

In [2]:
# Load the CSV file into a DataFrame
df = pd.read_csv("scraped bus details/df.jk.csv")

# Display the DataFrame
df

,Route_name,Route_link
0,Delhi to Srinagar,https://www.redbus.in/bus-tickets/delhi-to-sri...
1,Jammu (j and k) to Katra (jammu and kashmir),https://www.redbus.in/bus-tickets/jammu-to-katra
2,Katra (jammu and kashmir) to Jammu (j and k),https://www.redbus.in/bus-tickets/katra-to-jammu


In [3]:

driver_k = webdriver.Chrome()  # Initialize the Chrome WebDriver
Bus_names_k = []  # List to store bus names
Bus_types_k = []  # List to store bus types
Departure_k = []  # List to store departure times
Arrival_k = []  # List to store arrival times
Ratings_k = []  # List to store ratings
Total_Duration_k = []  # List to store total travel duration
Prices_k = []  # List to store prices
Seats_Available_k = []  # List to store seat availability
Route_names = []  # List to store route names
Route_links = []  # List to store route links

# Loop through each row of the DataFrame df
for i, r in df.iterrows():  
    link = r["Route_link"]  # Get the route link from the DataFrame
    routes = r["Route_name"]  # Get the route name from the DataFrame

    # Open the route link in the browser
    driver_k.get(link)  
    time.sleep(2)  # Wait for 2 seconds for the page to load

    # Find clickable elements that contain the route link
    elements = driver_k.find_elements(By.XPATH, f"//a[contains(@href, '{link}')]")  
    for element in elements:
        element.click()  # Click the element to reveal bus details
        time.sleep(2)  # Wait for 2 seconds for the page to update

    # Try clicking the "View Buses" button if available
    try:
        clicks = driver_k.find_element(By.XPATH, "//div[@class='button']")
        clicks.click()  # Click the button to view buses
    except:
        continue  # Skip to the next link if the button is not found
    time.sleep(2)  # Wait for the page to update

    scrolling = True  # Initialize a flag to start scrolling
    while scrolling:
        old_page_source = driver_k.page_source  # Store the current page source

        # Scroll down the page using the PAGE_DOWN key
        ActionChains(driver_k).send_keys(Keys.PAGE_DOWN).perform()  
        time.sleep(5)  # Wait for 5 seconds for the page to load more content

        new_page_source = driver_k.page_source  # Get the new page source

        # Stop scrolling if no new content is loaded
        if new_page_source == old_page_source:
            scrolling = False

    # Extract bus details such as name, type, departure time, etc.
    bus_name = driver_k.find_elements(By.XPATH, "//div[@class='travels lh-24 f-bold d-color']")
    bus_type = driver_k.find_elements(By.XPATH, "//div[@class='bus-type f-12 m-top-16 l-color evBus']")
    start_time = driver_k.find_elements(By.XPATH, "//*[@class='dp-time f-19 d-color f-bold']")
    end_time = driver_k.find_elements(By.XPATH, "//*[@class='bp-time f-19 d-color disp-Inline']")
    total_duration = driver_k.find_elements(By.XPATH, "//*[@class='dur l-color lh-24']")
    price = driver_k.find_elements(By.XPATH, '//div[@class="fare d-block"]//span')
    seats = driver_k.find_elements(By.XPATH, "//div[contains(@class, 'seat-left')]")

    # Attempt to extract ratings (if available)
    try:
        rating = driver_k.find_elements(By.XPATH, "//div[@class='clearfix row-one']/div[@class='column-six p-right-10 w-10 fl']")
    except:
        continue  # Skip to the next link if no ratings are found

    # Append extracted details to respective lists
    for bus in bus_name:
        Bus_names_k.append(bus.text)  # Append bus name to the list
        Route_links.append(link)  # Append route link to the list
        Route_names.append(routes)  # Append route name to the list
    for bus_type_elem in bus_type:
        Bus_types_k.append(bus_type_elem.text)  # Append bus type to the list
    for start_time_elem in start_time:
        Departure_k.append(start_time_elem.text)  # Append departure time to the list
    for end_time_elem in end_time:
        Arrival_k.append(end_time_elem.text)  # Append arrival time to the list
    for total_duration_elem in total_duration:
        Total_Duration_k.append(total_duration_elem.text)  # Append total duration to the list
    for ratings_elem in rating:
        Ratings_k.append(ratings_elem.text)  # Append ratings to the list
    for price_elem in price:
        Prices_k.append(price_elem.text)  # Append price to the list
    for seats_elem in seats:
        Seats_Available_k.append(seats_elem.text)  # Append available seats to the list

# Indicate successful completion
print("Successfully Completed")


Successfully Completed


In [15]:
# Create a dictionary where each key corresponds to a column name, and the value is the list of bus details
data = {
    'Bus_name': Bus_names_k,  # List containing the names of buses
    'Bus_type': Bus_types_k,  # List containing the types of buses (e.g., Sleeper, AC, etc.)
    'Start_time': Departure_k,  # List containing the departure times of the buses
    'End_time': Arrival_k,  # List containing the arrival times of the buses
    'Total_duration': Total_Duration_k,  # List containing the total travel duration for each bus
    'Price': Prices_k,  # List containing the price of each bus ticket
    "Seats_Available": Seats_Available_k,  # List containing the number of seats available for each bus
    "Ratings": Ratings_k,  # List containing the ratings for each bus
    'Route_name': Route_names,  # List containing the name of the routes (e.g., "Kozhikode to Ernakulam")
    'Route_link': Route_links  # List containing the URL links for each route
}

# Create a pandas DataFrame from the dictionary 'data' where each key-value pair becomes a column in the DataFrame
df_jk = pd.DataFrame(data)

# Define the path where the CSV file will be saved
path = r"/Users/priyankamalavade/Desktop/redbus_scraping/scraped_all_required_bus_details/df_jk.csv"

# Convert the DataFrame 'df_jk' into a CSV file and save it at the specified path
# index=False ensures that the row index is not included in the CSV file
df_jk.to_csv(path, index=False)

# Print a message indicating that the process has been completed successfully
print("Successfully Completed")

Successfully Completed


In [16]:
df_jk

,Bus_name,Bus_type,Start_time,End_time,Total_duration,Price,Seats_Available,Ratings,Route_name,Route_link


## RAJASTAN

In [16]:
# Load the CSV file into a DataFrame
df = pd.read_csv("scraped bus details/df.rajastan.csv")

# Display the DataFrame
df


,Route_name,Route_link
0,Jodhpur to Ajmer,https://www.redbus.in/bus-tickets/jodhpur-to-a...
1,Beawar (Rajasthan) to Jaipur (Rajasthan),https://www.redbus.in/bus-tickets/beawer-to-ja...
2,Udaipur to Jodhpur,https://www.redbus.in/bus-tickets/udaipur-to-j...
3,Jaipur (Rajasthan) to Jodhpur,https://www.redbus.in/bus-tickets/jaipur-to-jo...
4,Sikar to Jaipur (Rajasthan),https://www.redbus.in/bus-tickets/sikar-to-jaipur
5,Aligarh (uttar pradesh) to Jaipur (Rajasthan),https://www.redbus.in/bus-tickets/aligarh-utta...
6,Kota(Rajasthan) to Jaipur (Rajasthan),https://www.redbus.in/bus-tickets/kota-rajasth...
7,Jaipur (Rajasthan) to Pilani,https://www.redbus.in/bus-tickets/jaipur-to-pi...
8,Jaipur (Rajasthan) to Aligarh (uttar pradesh),https://www.redbus.in/bus-tickets/jaipur-to-al...
9,Kishangarh to Jaipur (Rajasthan),https://www.redbus.in/bus-tickets/kishangarh-t...


In [17]:
driver_k = webdriver.Chrome()  # Initialize the Chrome WebDriver
Bus_names_k = []  # List to store bus names
Bus_types_k = []  # List to store bus types
Departure_k = []  # List to store departure times
Arrival_k = []  # List to store arrival times
Ratings_k = []  # List to store ratings
Total_Duration_k = []  # List to store total travel duration
Prices_k = []  # List to store prices
Seats_Available_k = []  # List to store seat availability
Route_names = []  # List to store route names
Route_links = []  # List to store route links

# Loop through each row of the DataFrame df
for i, r in df.iterrows():  
    link = r["Route_link"]  # Get the route link from the DataFrame
    routes = r["Route_name"]  # Get the route name from the DataFrame

    # Open the route link in the browser
    driver_k.get(link)  
    time.sleep(2)  # Wait for 2 seconds for the page to load

    # Find clickable elements that contain the route link
    elements = driver_k.find_elements(By.XPATH, f"//a[contains(@href, '{link}')]")  
    for element in elements:
        element.click()  # Click the element to reveal bus details
        time.sleep(2)  # Wait for 2 seconds for the page to update

    # Try clicking the "View Buses" button if available
    try:
        clicks = driver_k.find_element(By.XPATH, "//div[@class='button']")
        clicks.click()  # Click the button to view buses
    except:
        continue  # Skip to the next link if the button is not found
    time.sleep(2)  # Wait for the page to update

    scrolling = True  # Initialize a flag to start scrolling
    while scrolling:
        old_page_source = driver_k.page_source  # Store the current page source

        # Scroll down the page using the PAGE_DOWN key
        ActionChains(driver_k).send_keys(Keys.PAGE_DOWN).perform()  
        time.sleep(5)  # Wait for 5 seconds for the page to load more content

        new_page_source = driver_k.page_source  # Get the new page source

        # Stop scrolling if no new content is loaded
        if new_page_source == old_page_source:
            scrolling = False

    # Extract bus details such as name, type, departure time, etc.
    bus_name = driver_k.find_elements(By.XPATH, "//div[@class='travels lh-24 f-bold d-color']")
    bus_type = driver_k.find_elements(By.XPATH, "//div[@class='bus-type f-12 m-top-16 l-color evBus']")
    start_time = driver_k.find_elements(By.XPATH, "//*[@class='dp-time f-19 d-color f-bold']")
    end_time = driver_k.find_elements(By.XPATH, "//*[@class='bp-time f-19 d-color disp-Inline']")
    total_duration = driver_k.find_elements(By.XPATH, "//*[@class='dur l-color lh-24']")
    price = driver_k.find_elements(By.XPATH, '//div[@class="fare d-block"]//span')
    seats = driver_k.find_elements(By.XPATH, "//div[contains(@class, 'seat-left')]")

    # Attempt to extract ratings (if available)
    try:
        rating = driver_k.find_elements(By.XPATH, "//div[@class='clearfix row-one']/div[@class='column-six p-right-10 w-10 fl']")
    except:
        continue  # Skip to the next link if no ratings are found

    # Append extracted details to respective lists
    for bus in bus_name:
        Bus_names_k.append(bus.text)  # Append bus name to the list
        Route_links.append(link)  # Append route link to the list
        Route_names.append(routes)  # Append route name to the list
    for bus_type_elem in bus_type:
        Bus_types_k.append(bus_type_elem.text)  # Append bus type to the list
    for start_time_elem in start_time:
        Departure_k.append(start_time_elem.text)  # Append departure time to the list
    for end_time_elem in end_time:
        Arrival_k.append(end_time_elem.text)  # Append arrival time to the list
    for total_duration_elem in total_duration:
        Total_Duration_k.append(total_duration_elem.text)  # Append total duration to the list
    for ratings_elem in rating:
        Ratings_k.append(ratings_elem.text)  # Append ratings to the list
    for price_elem in price:
        Prices_k.append(price_elem.text)  # Append price to the list
    for seats_elem in seats:
        Seats_Available_k.append(seats_elem.text)  # Append available seats to the list

# Indicate successful completion
print("Successfully Completed")


Successfully Completed


In [18]:
# Create a dictionary where each key corresponds to a column name, and the value is the list of bus details
data = {
    'Bus_name': Bus_names_k,  # List containing the names of buses
    'Bus_type': Bus_types_k,  # List containing the types of buses (e.g., Sleeper, AC, etc.)
    'Start_time': Departure_k,  # List containing the departure times of the buses
    'End_time': Arrival_k,  # List containing the arrival times of the buses
    'Total_duration': Total_Duration_k,  # List containing the total travel duration for each bus
    'Price': Prices_k,  # List containing the price of each bus ticket
    "Seats_Available": Seats_Available_k,  # List containing the number of seats available for each bus
    "Ratings": Ratings_k,  # List containing the ratings for each bus
    'Route_name': Route_names,  # List containing the name of the routes (e.g., "Kozhikode to Ernakulam")
    'Route_link': Route_links  # List containing the URL links for each route
}

# Create a pandas DataFrame from the dictionary 'data' where each key-value pair becomes a column in the DataFrame
df_rajastan = pd.DataFrame(data)

# Define the path where the CSV file will be saved
path = r"/Users/priyankamalavade/Desktop/redbus_scraping/scraped_all_required_bus_details/df_rajastan.csv"


# Convert the DataFrame 'df_rajastan' into a CSV file and save it at the specified path
# index=False ensures that the row index is not included in the CSV file
df_rajastan.to_csv(path, index=False)

# Print a message indicating that the process has been completed successfully
print("Successfully Completed")


Successfully Completed


In [19]:
df_rajastan

,Bus_name,Bus_type,Start_time,End_time,Total_duration,Price,Seats_Available,Ratings,Route_name,Route_link
0,RSRTC - 189473,NON AC Seater / Sleeper 2+1,00:45,04:30,03h 45m,282,29 Seats available,3.6,Beawar (Rajasthan) to Jaipur (Rajasthan),https://www.redbus.in/bus-tickets/beawer-to-ja...
1,RSRTC - 179810,NON AC Seater / Sleeper 2+1,00:45,04:45,04h 00m,282,16 Seats available,3.6,Beawar (Rajasthan) to Jaipur (Rajasthan),https://www.redbus.in/bus-tickets/beawer-to-ja...
2,RSRTC - 193312,A/C Seater / Sleeper (2+1),01:15,05:00,03h 45m,413,17 Seats available,3.6,Beawar (Rajasthan) to Jaipur (Rajasthan),https://www.redbus.in/bus-tickets/beawer-to-ja...
3,RSRTC - 192299,Super Luxury Volvo AC Seater Pushback 2+2,01:45,05:30,03h 45m,424,37 Seats available,4.7\n7,Beawar (Rajasthan) to Jaipur (Rajasthan),https://www.redbus.in/bus-tickets/beawer-to-ja...
4,RSRTC - 189583,NON AC Seater / Sleeper 2+1,02:45,06:30,03h 45m,282,37 Seats available,3.6,Beawar (Rajasthan) to Jaipur (Rajasthan),https://www.redbus.in/bus-tickets/beawer-to-ja...
...,...,...,...,...,...,...,...,...,...,...
190,Raj travels,A/C Sleeper (2+1),22:15,06:30,08h 15m,499,13 Seats available,4.8\n472,Udaipur to Jaipur (Rajasthan),https://www.redbus.in/bus-tickets/udaipur-to-j...
191,Pooja Yaduvanshi Travels,A/C Seater / Sleeper (2+1),22:00,06:00,08h 00m,806,7 Seats available,4.8\n455,Udaipur to Jaipur (Rajasthan),https://www.redbus.in/bus-tickets/udaipur-to-j...
192,Lake City Travels,AC Sleeper (2+1),22:15,07:05,08h 50m,699,17 Seats available,4.7\n408,Udaipur to Jaipur (Rajasthan),https://www.redbus.in/bus-tickets/udaipur-to-j...
193,SURYAVANSHI TRAVELS,A/C Sleeper (2+1),22:00,06:15,08h 15m,750,13 Seats available,4.6\n418,Udaipur to Jaipur (Rajasthan),https://www.redbus.in/bus-tickets/udaipur-to-j...


## SOUTH BENGAL

In [21]:
 #Load the CSV file into a DataFrame
df = pd.read_csv("scraped bus details/df.south_bengal.csv")

# Display the DataFrame
df


,Route_name,Route_link
0,Durgapur (West Bengal) to Kolkata,https://www.redbus.in/bus-tickets/durgapur-to-...
1,Kolkata to Burdwan,https://www.redbus.in/bus-tickets/kolkata-to-b...
2,Kolkata to Durgapur (West Bengal),https://www.redbus.in/bus-tickets/kolkata-to-d...
3,Haldia to Kolkata,https://www.redbus.in/bus-tickets/haldia-to-ko...
4,Kolkata to Haldia,https://www.redbus.in/bus-tickets/kolkata-to-h...
5,Midnapore to Kolkata,https://www.redbus.in/bus-tickets/midnapore-to...
6,Kolkata to Arambagh (West Bengal),https://www.redbus.in/bus-tickets/kolkata-to-a...
7,Kolkata to Digha,https://www.redbus.in/bus-tickets/kolkata-to-d...
8,Digha to Kolkata,https://www.redbus.in/bus-tickets/digha-to-kol...
9,Kolkata to Bankura,https://www.redbus.in/bus-tickets/kolkata-to-b...


In [22]:
driver_k = webdriver.Chrome()  # Initialize the Chrome WebDriver
Bus_names_k = []  # List to store bus names
Bus_types_k = []  # List to store bus types
Departure_k = []  # List to store departure times
Arrival_k = []  # List to store arrival times
Ratings_k = []  # List to store ratings
Total_Duration_k = []  # List to store total travel duration
Prices_k = []  # List to store prices
Seats_Available_k = []  # List to store seat availability
Route_names = []  # List to store route names
Route_links = []  # List to store route links

# Loop through each row of the DataFrame df
for i, r in df.iterrows():  
    link = r["Route_link"]  # Get the route link from the DataFrame
    routes = r["Route_name"]  # Get the route name from the DataFrame

    # Open the route link in the browser
    driver_k.get(link)  
    time.sleep(2)  # Wait for 2 seconds for the page to load

    # Find clickable elements that contain the route link
    elements = driver_k.find_elements(By.XPATH, f"//a[contains(@href, '{link}')]")  
    for element in elements:
        element.click()  # Click the element to reveal bus details
        time.sleep(2)  # Wait for 2 seconds for the page to update

    # Try clicking the "View Buses" button if available
    try:
        clicks = driver_k.find_element(By.XPATH, "//div[@class='button']")
        clicks.click()  # Click the button to view buses
    except:
        continue  # Skip to the next link if the button is not found
    time.sleep(2)  # Wait for the page to update

    scrolling = True  # Initialize a flag to start scrolling
    while scrolling:
        old_page_source = driver_k.page_source  # Store the current page source

        # Scroll down the page using the PAGE_DOWN key
        ActionChains(driver_k).send_keys(Keys.PAGE_DOWN).perform()  
        time.sleep(5)  # Wait for 5 seconds for the page to load more content

        new_page_source = driver_k.page_source  # Get the new page source

        # Stop scrolling if no new content is loaded
        if new_page_source == old_page_source:
            scrolling = False

    # Extract bus details such as name, type, departure time, etc.
    bus_name = driver_k.find_elements(By.XPATH, "//div[@class='travels lh-24 f-bold d-color']")
    bus_type = driver_k.find_elements(By.XPATH, "//div[@class='bus-type f-12 m-top-16 l-color evBus']")
    start_time = driver_k.find_elements(By.XPATH, "//*[@class='dp-time f-19 d-color f-bold']")
    end_time = driver_k.find_elements(By.XPATH, "//*[@class='bp-time f-19 d-color disp-Inline']")
    total_duration = driver_k.find_elements(By.XPATH, "//*[@class='dur l-color lh-24']")
    price = driver_k.find_elements(By.XPATH, '//div[@class="fare d-block"]//span')
    seats = driver_k.find_elements(By.XPATH, "//div[contains(@class, 'seat-left')]")

    # Attempt to extract ratings (if available)
    try:
        rating = driver_k.find_elements(By.XPATH, "//div[@class='clearfix row-one']/div[@class='column-six p-right-10 w-10 fl']")
    except:
        continue  # Skip to the next link if no ratings are found

    # Append extracted details to respective lists
    for bus in bus_name:
        Bus_names_k.append(bus.text)  # Append bus name to the list
        Route_links.append(link)  # Append route link to the list
        Route_names.append(routes)  # Append route name to the list
    for bus_type_elem in bus_type:
        Bus_types_k.append(bus_type_elem.text)  # Append bus type to the list
    for start_time_elem in start_time:
        Departure_k.append(start_time_elem.text)  # Append departure time to the list
    for end_time_elem in end_time:
        Arrival_k.append(end_time_elem.text)  # Append arrival time to the list
    for total_duration_elem in total_duration:
        Total_Duration_k.append(total_duration_elem.text)  # Append total duration to the list
    for ratings_elem in rating:
        Ratings_k.append(ratings_elem.text)  # Append ratings to the list
    for price_elem in price:
        Prices_k.append(price_elem.text)  # Append price to the list
    for seats_elem in seats:
        Seats_Available_k.append(seats_elem.text)  # Append available seats to the list

# Indicate successful completion
print("Successfully Completed")


Successfully Completed


In [23]:
# Create a dictionary where each key corresponds to a column name, and the value is the list of bus details
data = {
    'Bus_name': Bus_names_k,  # List containing the names of buses
    'Bus_type': Bus_types_k,  # List containing the types of buses (e.g., Sleeper, AC, etc.)
    'Start_time': Departure_k,  # List containing the departure times of the buses
    'End_time': Arrival_k,  # List containing the arrival times of the buses
    'Total_duration': Total_Duration_k,  # List containing the total travel duration for each bus
    'Price': Prices_k,  # List containing the price of each bus ticket
    "Seats_Available": Seats_Available_k,  # List containing the number of seats available for each bus
    "Ratings": Ratings_k,  # List containing the ratings for each bus
    'Route_name': Route_names,  # List containing the name of the routes (e.g., "Kozhikode to Ernakulam")
    'Route_link': Route_links  # List containing the URL links for each route
}

# Create a pandas DataFrame from the dictionary 'data' where each key-value pair becomes a column in the DataFrame
df_south_bengal = pd.DataFrame(data)

# Define the path where the CSV file will be saved
path = r"/Users/priyankamalavade/Desktop/redbus_scraping/scraped_all_required_bus_details/df_south_bengal.csv"


# Convert the DataFrame 'df_south_bengal' into a CSV file and save it at the specified path
# index=False ensures that the row index is not included in the CSV file
df_south_bengal.to_csv(path, index=False)

# Print a message indicating that the process has been completed successfully
print("Successfully Completed")


Successfully Completed


In [24]:
df_south_bengal

,Bus_name,Bus_type,Start_time,End_time,Total_duration,Price,Seats_Available,Ratings,Route_name,Route_link
0,SBSTC-DURGAPUR - KOLKATA - 05:00 (DGP DEPOT) -...,Non AC Seater (2+3),05:00,08:30,03h 30m,150,40 Seats available,4.2\n166,Durgapur (West Bengal) to Kolkata,https://www.redbus.in/bus-tickets/durgapur-to-...
1,SBSTC-DURGAPUR - KARUNAMOYEE - 05:15 (DGP DEPO...,Non AC Seater (2+3),05:15,09:15,04h 00m,155,22 Seats available,4.2\n290,Durgapur (West Bengal) to Kolkata,https://www.redbus.in/bus-tickets/durgapur-to-...
2,SBSTC-DURGAPUR - KOLKATA - 05:30 (DGP DEPOT) -...,Non AC Seater (2+3),05:30,09:00,03h 30m,150,47 Seats available,3.8\n88,Durgapur (West Bengal) to Kolkata,https://www.redbus.in/bus-tickets/durgapur-to-...
3,SBSTC-DURGAPUR - KOLKATA - L/S - 06:00 (DGP DE...,Non AC Seater (2+3),06:00,09:30,03h 30m,150,29 Seats available,3.5\n96,Durgapur (West Bengal) to Kolkata,https://www.redbus.in/bus-tickets/durgapur-to-...
4,SBSTC-DURGAPUR - KARUNAMOYEE - 06:15 (DGP DEPO...,Non AC Seater (2+3),06:15,10:15,04h 00m,155,22 Seats available,4.0\n237,Durgapur (West Bengal) to Kolkata,https://www.redbus.in/bus-tickets/durgapur-to-...
...,...,...,...,...,...,...,...,...,...,...
228,,,,,,,,,Siliguri to Kolkata,https://www.redbus.in/bus-tickets/siliguri-to-...
229,,,,,,,,,Siliguri to Kolkata,https://www.redbus.in/bus-tickets/siliguri-to-...
230,,,,,,,,,Siliguri to Kolkata,https://www.redbus.in/bus-tickets/siliguri-to-...
231,,,,,,,,,Siliguri to Kolkata,https://www.redbus.in/bus-tickets/siliguri-to-...


## TELANGANA

In [3]:
# Load the CSV file into a DataFrame
df = pd.read_csv("scraped bus details/df.telangana.csv")

# Display the DataFrame
df


,Route_name,Route_link
0,Khammam to Hyderabad,https://www.redbus.in/bus-tickets/khammam-to-h...
1,Hyderabad to Vijayawada,https://www.redbus.in/bus-tickets/hyderabad-to...
2,Hyderabad to Khammam,https://www.redbus.in/bus-tickets/hyderabad-to...
3,Hyderabad to Srisailam,https://www.redbus.in/bus-tickets/hyderabad-to...
4,Karimnagar to Hyderabad,https://www.redbus.in/bus-tickets/karimnagar-t...
5,Hyderabad to Adilabad,https://www.redbus.in/bus-tickets/hyderabad-to...
6,Kothagudem to Hyderabad,https://www.redbus.in/bus-tickets/kothagudem-t...
7,Hyderabad to Mancherial,https://www.redbus.in/bus-tickets/hyderabad-to...
8,Guntur (Andhra Pradesh) to Hyderabad,https://www.redbus.in/bus-tickets/guntur-to-hy...
9,Godavarikhani to Hyderabad,https://www.redbus.in/bus-tickets/godavarikhan...


In [6]:
driver_k = webdriver.Chrome()  # Initialize the Chrome WebDriver
Bus_names_k = []  # List to store bus names
Bus_types_k = []  # List to store bus types
Departure_k = []  # List to store departure times
Arrival_k = []  # List to store arrival times
Ratings_k = []  # List to store ratings
Total_Duration_k = []  # List to store total travel duration
Prices_k = []  # List to store prices
Seats_Available_k = []  # List to store seat availability
Route_names = []  # List to store route names
Route_links = []  # List to store route links

# Loop through each row of the DataFrame df
for i, r in df.iterrows():  
    link = r["Route_link"]  # Get the route link from the DataFrame
    routes = r["Route_name"]  # Get the route name from the DataFrame

    # Open the route link in the browser
    driver_k.get(link)  
    time.sleep(2)  # Wait for 2 seconds for the page to load

    # Find clickable elements that contain the route link
    elements = driver_k.find_elements(By.XPATH, f"//a[contains(@href, '{link}')]")  
    for element in elements:
        element.click()  # Click the element to reveal bus details
        time.sleep(2)  # Wait for 2 seconds for the page to update

    # Try clicking the "View Buses" button if available
    try:
        clicks = driver_k.find_element(By.XPATH, "//div[@class='button']")
        clicks.click()  # Click the button to view buses
    except:
        continue  # Skip to the next link if the button is not found
    time.sleep(2)  # Wait for the page to update

    scrolling = True  # Initialize a flag to start scrolling
    while scrolling:
        old_page_source = driver_k.page_source  # Store the current page source

        # Scroll down the page using the PAGE_DOWN key
        ActionChains(driver_k).send_keys(Keys.PAGE_DOWN).perform()  
        time.sleep(5)  # Wait for 5 seconds for the page to load more content

        new_page_source = driver_k.page_source  # Get the new page source

        # Stop scrolling if no new content is loaded
        if new_page_source == old_page_source:
            scrolling = False

    # Extract bus details such as name, type, departure time, etc.
    bus_name = driver_k.find_elements(By.XPATH, "//div[@class='travels lh-24 f-bold d-color']")
    bus_type = driver_k.find_elements(By.XPATH, "//div[@class='bus-type f-12 m-top-16 l-color evBus']")
    start_time = driver_k.find_elements(By.XPATH, "//*[@class='dp-time f-19 d-color f-bold']")
    end_time = driver_k.find_elements(By.XPATH, "//*[@class='bp-time f-19 d-color disp-Inline']")
    total_duration = driver_k.find_elements(By.XPATH, "//*[@class='dur l-color lh-24']")
    price = driver_k.find_elements(By.XPATH, '//div[@class="fare d-block"]//span')
    seats = driver_k.find_elements(By.XPATH, "//div[contains(@class, 'seat-left')]")

    # Attempt to extract ratings (if available)
    try:
        rating = driver_k.find_elements(By.XPATH, "//div[@class='clearfix row-one']/div[@class='column-six p-right-10 w-10 fl']")
    except:
        continue  # Skip to the next link if no ratings are found

    # Append extracted details to respective lists
    for bus in bus_name:
        Bus_names_k.append(bus.text)  # Append bus name to the list
        Route_links.append(link)  # Append route link to the list
        Route_names.append(routes)  # Append route name to the list
    for bus_type_elem in bus_type:
        Bus_types_k.append(bus_type_elem.text)  # Append bus type to the list
    for start_time_elem in start_time:
        Departure_k.append(start_time_elem.text)  # Append departure time to the list
    for end_time_elem in end_time:
        Arrival_k.append(end_time_elem.text)  # Append arrival time to the list
    for total_duration_elem in total_duration:
        Total_Duration_k.append(total_duration_elem.text)  # Append total duration to the list
    for ratings_elem in rating:
        Ratings_k.append(ratings_elem.text)  # Append ratings to the list
    for price_elem in price:
        Prices_k.append(price_elem.text)  # Append price to the list
    for seats_elem in seats:
        Seats_Available_k.append(seats_elem.text)  # Append available seats to the list

# Indicate successful completion
print("Successfully Completed")


Successfully Completed


In [9]:
# Create a dictionary where each key corresponds to a column name, and the value is the list of bus details
data = {
    'Bus_name': Bus_names_k,  # List containing the names of buses
    'Bus_type': Bus_types_k,  # List containing the types of buses (e.g., Sleeper, AC, etc.)
    'Start_time': Departure_k,  # List containing the departure times of the buses
    'End_time': Arrival_k,  # List containing the arrival times of the buses
    'Total_duration': Total_Duration_k,  # List containing the total travel duration for each bus
    'Price': Prices_k,  # List containing the price of each bus ticket
    "Seats_Available": Seats_Available_k,  # List containing the number of seats available for each bus
    "Ratings": Ratings_k,  # List containing the ratings for each bus
    'Route_name': Route_names,  # List containing the name of the routes (e.g., "Kozhikode to Ernakulam")
    'Route_link': Route_links  # List containing the URL links for each route
}

# Create a pandas DataFrame from the dictionary 'data' where each key-value pair becomes a column in the DataFrame
df_telangana = pd.DataFrame(data)

# Define the path where the CSV file will be saved
path = r"/Users/priyankamalavade/Desktop/redbus_scraping/scraped_all_required_bus_details/df_telangana.csv"


# Convert the DataFrame 'df_telangana' into a CSV file and save it at the specified path
# index=False ensures that the row index is not included in the CSV file
df_telangana.to_csv(path, index=False)

# Print a message indicating that the process has been completed successfully
print("Successfully Completed")


Successfully Completed


In [10]:
df_telangana

,Bus_name,Bus_type,Start_time,End_time,Total_duration,Price,Seats_Available,Ratings,Route_name,Route_link
0,Bharathi Tours & Travels,A/C Sleeper (2+1),00:50,05:30,04h 40m,590,1 Seat available,4.5,Khammam to Hyderabad,https://www.redbus.in/bus-tickets/khammam-to-h...
1,Sri Tulasi Tours and Travels,A/C Sleeper (2+1),01:00,04:30,03h 30m,599,4 Seats available,3.6,Khammam to Hyderabad,https://www.redbus.in/bus-tickets/khammam-to-h...
2,Sri Rama Travels,A/C Seater / Sleeper (2+1),01:30,06:45,05h 15m,777,2 Seats available,4.1\n4,Khammam to Hyderabad,https://www.redbus.in/bus-tickets/khammam-to-h...
3,Mahadevv travels,Non A/C Seater / Sleeper (2+1),02:20,05:30,03h 10m,499,2 Seats available,3.4,Khammam to Hyderabad,https://www.redbus.in/bus-tickets/khammam-to-h...
4,Bharathi Tours & Travels,Non A/C Seater / Sleeper (2+1),23:40,04:00,04h 20m,711,27 Seats available,4.5\n12,Khammam to Hyderabad,https://www.redbus.in/bus-tickets/khammam-to-h...
...,...,...,...,...,...,...,...,...,...,...
192,,,,,,,,,Hyderabad to Kodad,https://www.redbus.in/bus-tickets/hyderabad-to...
193,,,,,,,,,Hyderabad to Kodad,https://www.redbus.in/bus-tickets/hyderabad-to...
194,,,,,,,,,Hyderabad to Kodad,https://www.redbus.in/bus-tickets/hyderabad-to...
195,,,,,,,,,Hyderabad to Kodad,https://www.redbus.in/bus-tickets/hyderabad-to...



## UTTAR PRADESH

In [11]:
# Load the CSV file into a DataFrame
df = pd.read_csv("scraped bus details/df.up.csv")

# Display the DataFrame
df



,Route_name,Route_link
0,Delhi to Bareilly (Uttar Pradesh),https://www.redbus.in/bus-tickets/delhi-to-bar...
1,Delhi to Lucknow,https://www.redbus.in/bus-tickets/delhi-to-luc...
2,Bareilly (Uttar Pradesh) to Delhi,https://www.redbus.in/bus-tickets/bareilly-to-...
3,Lucknow to Gorakhpur (uttar pradesh),https://www.redbus.in/bus-tickets/lucknow-to-g...
4,Aligarh (uttar pradesh) to Delhi,https://www.redbus.in/bus-tickets/aligarh-utta...
5,Lucknow to Prayagraj(Uttar Pradesh),https://www.redbus.in/bus-tickets/lucknow-to-a...
6,Delhi to Aligarh (uttar pradesh),https://www.redbus.in/bus-tickets/delhi-to-ali...
7,Lucknow to Delhi,https://www.redbus.in/bus-tickets/lucknow-to-d...
8,Prayagraj(Uttar Pradesh) to Lucknow,https://www.redbus.in/bus-tickets/allahabad-to...
9,Delhi to Farrukhabad (Uttar Pradesh),https://www.redbus.in/bus-tickets/delhi-to-far...


In [12]:
driver_k = webdriver.Chrome()  # Initialize the Chrome WebDriver
Bus_names_k = []  # List to store bus names
Bus_types_k = []  # List to store bus types
Departure_k = []  # List to store departure times
Arrival_k = []  # List to store arrival times
Ratings_k = []  # List to store ratings
Total_Duration_k = []  # List to store total travel duration
Prices_k = []  # List to store prices
Seats_Available_k = []  # List to store seat availability
Route_names = []  # List to store route names
Route_links = []  # List to store route links

# Loop through each row of the DataFrame df
for i, r in df.iterrows():  
    link = r["Route_link"]  # Get the route link from the DataFrame
    routes = r["Route_name"]  # Get the route name from the DataFrame

    # Open the route link in the browser
    driver_k.get(link)  
    time.sleep(2)  # Wait for 2 seconds for the page to load

    # Find clickable elements that contain the route link
    elements = driver_k.find_elements(By.XPATH, f"//a[contains(@href, '{link}')]")  
    for element in elements:
        element.click()  # Click the element to reveal bus details
        time.sleep(2)  # Wait for 2 seconds for the page to update

    # Try clicking the "View Buses" button if available
    try:
        clicks = driver_k.find_element(By.XPATH, "//div[@class='button']")
        clicks.click()  # Click the button to view buses
    except:
        continue  # Skip to the next link if the button is not found
    time.sleep(2)  # Wait for the page to update

    scrolling = True  # Initialize a flag to start scrolling
    while scrolling:
        old_page_source = driver_k.page_source  # Store the current page source

        # Scroll down the page using the PAGE_DOWN key
        ActionChains(driver_k).send_keys(Keys.PAGE_DOWN).perform()  
        time.sleep(5)  # Wait for 5 seconds for the page to load more content

        new_page_source = driver_k.page_source  # Get the new page source

        # Stop scrolling if no new content is loaded
        if new_page_source == old_page_source:
            scrolling = False

    # Extract bus details such as name, type, departure time, etc.
    bus_name = driver_k.find_elements(By.XPATH, "//div[@class='travels lh-24 f-bold d-color']")
    bus_type = driver_k.find_elements(By.XPATH, "//div[@class='bus-type f-12 m-top-16 l-color evBus']")
    start_time = driver_k.find_elements(By.XPATH, "//*[@class='dp-time f-19 d-color f-bold']")
    end_time = driver_k.find_elements(By.XPATH, "//*[@class='bp-time f-19 d-color disp-Inline']")
    total_duration = driver_k.find_elements(By.XPATH, "//*[@class='dur l-color lh-24']")
    price = driver_k.find_elements(By.XPATH, '//div[@class="fare d-block"]//span')
    seats = driver_k.find_elements(By.XPATH, "//div[contains(@class, 'seat-left')]")

    # Attempt to extract ratings (if available)
    try:
        rating = driver_k.find_elements(By.XPATH, "//div[@class='clearfix row-one']/div[@class='column-six p-right-10 w-10 fl']")
    except:
        continue  # Skip to the next link if no ratings are found

    # Append extracted details to respective lists
    for bus in bus_name:
        Bus_names_k.append(bus.text)  # Append bus name to the list
        Route_links.append(link)  # Append route link to the list
        Route_names.append(routes)  # Append route name to the list
    for bus_type_elem in bus_type:
        Bus_types_k.append(bus_type_elem.text)  # Append bus type to the list
    for start_time_elem in start_time:
        Departure_k.append(start_time_elem.text)  # Append departure time to the list
    for end_time_elem in end_time:
        Arrival_k.append(end_time_elem.text)  # Append arrival time to the list
    for total_duration_elem in total_duration:
        Total_Duration_k.append(total_duration_elem.text)  # Append total duration to the list
    for ratings_elem in rating:
        Ratings_k.append(ratings_elem.text)  # Append ratings to the list
    for price_elem in price:
        Prices_k.append(price_elem.text)  # Append price to the list
    for seats_elem in seats:
        Seats_Available_k.append(seats_elem.text)  # Append available seats to the list

# Indicate successful completion
print("Successfully Completed")


Successfully Completed


In [13]:
# Create a dictionary where each key corresponds to a column name, and the value is the list of bus details
data = {
    'Bus_name': Bus_names_k,  # List containing the names of buses
    'Bus_type': Bus_types_k,  # List containing the types of buses (e.g., Sleeper, AC, etc.)
    'Start_time': Departure_k,  # List containing the departure times of the buses
    'End_time': Arrival_k,  # List containing the arrival times of the buses
    'Total_duration': Total_Duration_k,  # List containing the total travel duration for each bus
    'Price': Prices_k,  # List containing the price of each bus ticket
    "Seats_Available": Seats_Available_k,  # List containing the number of seats available for each bus
    "Ratings": Ratings_k,  # List containing the ratings for each bus
    'Route_name': Route_names,  # List containing the name of the routes (e.g., "Kozhikode to Ernakulam")
    'Route_link': Route_links  # List containing the URL links for each route
}

# Create a pandas DataFrame from the dictionary 'data' where each key-value pair becomes a column in the DataFrame
df_up = pd.DataFrame(data)

# Define the path where the CSV file will be saved
path = r"/Users/priyankamalavade/Desktop/redbus_scraping/scraped_all_required_bus_details/df_up.csv"


# Convert the DataFrame 'df_up' into a CSV file and save it at the specified path
# index=False ensures that the row index is not included in the CSV file
df_up.to_csv(path, index=False)

# Print a message indicating that the process has been completed successfully
print("Successfully Completed")


Successfully Completed


In [14]:
df_up

,Bus_name,Bus_type,Start_time,End_time,Total_duration,Price,Seats_Available,Ratings,Route_name,Route_link
0,FlixBus,AC Seater (2+2),23:35,05:00,05h 25m,608.04,9 Seats available,4.3\n125,Delhi to Bareilly (Uttar Pradesh),https://www.redbus.in/bus-tickets/delhi-to-bar...
1,RS YADAV SMART BUS,Bharat Benz A/C Seater /Sleeper (2+1),21:40,04:45,07h 05m,1199,14 Seats available,3.9\n37,Delhi to Bareilly (Uttar Pradesh),https://www.redbus.in/bus-tickets/delhi-to-bar...
2,Vijay Tour and Travels,A/C Seater / Sleeper (2+1),01:40,06:20,04h 40m,849,18 Seats available,3.0\n74,Delhi to Bareilly (Uttar Pradesh),https://www.redbus.in/bus-tickets/delhi-to-bar...
3,New Khaira TPT,AC Sleeper (2+1),20:30,02:50,06h 20m,900,16 Seats available,4.3\n13,Delhi to Bareilly (Uttar Pradesh),https://www.redbus.in/bus-tickets/delhi-to-bar...
4,New Khaira Transport,A/C Seater / Sleeper (2+2),21:00,03:30,06h 30m,500,19 Seats available,1.9\n38,Delhi to Bareilly (Uttar Pradesh),https://www.redbus.in/bus-tickets/delhi-to-bar...
...,...,...,...,...,...,...,...,...,...,...
313,UPSRTC - SJP0200,Ordinary Non AC Seater 2+3,07:00,14:20,07h 20m,584,52 Seats available,3.3,Shahjahanpur (Uttar Pradesh) to Delhi,https://www.redbus.in/bus-tickets/shahjahanpur...
314,UPSRTC - SJP0203,Ordinary Non AC Seater 2+3,08:02,15:22,07h 20m,584,52 Seats available,3.3,Shahjahanpur (Uttar Pradesh) to Delhi,https://www.redbus.in/bus-tickets/shahjahanpur...
315,,,,,,,,,Shahjahanpur (Uttar Pradesh) to Delhi,https://www.redbus.in/bus-tickets/shahjahanpur...
316,,,,,,,,,Shahjahanpur (Uttar Pradesh) to Delhi,https://www.redbus.in/bus-tickets/shahjahanpur...


## WEST BENGAL

In [16]:
# Load the CSV file into a DataFrame
df = pd.read_csv("scraped bus details/df.wb.csv")

# Display the DataFrame
df



,Route_name,Route_link
0,Digha to Barasat (West Bengal),https://www.redbus.in/bus-tickets/digha-to-bar...
1,Durgapur (West Bengal) to Kolkata,https://www.redbus.in/bus-tickets/durgapur-to-...
2,Digha to Kolkata,https://www.redbus.in/bus-tickets/digha-to-kol...
3,Barasat (West Bengal) to Digha,https://www.redbus.in/bus-tickets/barasat-west...
4,Kolkata to Durgapur (West Bengal),https://www.redbus.in/bus-tickets/kolkata-to-d...
5,Kolkata to Digha,https://www.redbus.in/bus-tickets/kolkata-to-d...
6,Barasat (West Bengal) to Contai (Kanthi),https://www.redbus.in/bus-tickets/barasat-west...
7,Barasat (West Bengal) to Midnapore,https://www.redbus.in/bus-tickets/barasat-west...
8,Barasat (West Bengal) to Nandakumar (west bengal),https://www.redbus.in/bus-tickets/barasat-west...
9,Barasat (West Bengal) to Kolaghat,https://www.redbus.in/bus-tickets/barasat-west...


In [17]:
driver_k = webdriver.Chrome()  # Initialize the Chrome WebDriver
Bus_names_k = []  # List to store bus names
Bus_types_k = []  # List to store bus types
Departure_k = []  # List to store departure times
Arrival_k = []  # List to store arrival times
Ratings_k = []  # List to store ratings
Total_Duration_k = []  # List to store total travel duration
Prices_k = []  # List to store prices
Seats_Available_k = []  # List to store seat availability
Route_names = []  # List to store route names
Route_links = []  # List to store route links

# Loop through each row of the DataFrame df
for i, r in df.iterrows():  
    link = r["Route_link"]  # Get the route link from the DataFrame
    routes = r["Route_name"]  # Get the route name from the DataFrame

    # Open the route link in the browser
    driver_k.get(link)  
    time.sleep(2)  # Wait for 2 seconds for the page to load

    # Find clickable elements that contain the route link
    elements = driver_k.find_elements(By.XPATH, f"//a[contains(@href, '{link}')]")  
    for element in elements:
        element.click()  # Click the element to reveal bus details
        time.sleep(2)  # Wait for 2 seconds for the page to update

    # Try clicking the "View Buses" button if available
    try:
        clicks = driver_k.find_element(By.XPATH, "//div[@class='button']")
        clicks.click()  # Click the button to view buses
    except:
        continue  # Skip to the next link if the button is not found
    time.sleep(2)  # Wait for the page to update

    scrolling = True  # Initialize a flag to start scrolling
    while scrolling:
        old_page_source = driver_k.page_source  # Store the current page source

        # Scroll down the page using the PAGE_DOWN key
        ActionChains(driver_k).send_keys(Keys.PAGE_DOWN).perform()  
        time.sleep(5)  # Wait for 5 seconds for the page to load more content

        new_page_source = driver_k.page_source  # Get the new page source

        # Stop scrolling if no new content is loaded
        if new_page_source == old_page_source:
            scrolling = False

    # Extract bus details such as name, type, departure time, etc.
    bus_name = driver_k.find_elements(By.XPATH, "//div[@class='travels lh-24 f-bold d-color']")
    bus_type = driver_k.find_elements(By.XPATH, "//div[@class='bus-type f-12 m-top-16 l-color evBus']")
    start_time = driver_k.find_elements(By.XPATH, "//*[@class='dp-time f-19 d-color f-bold']")
    end_time = driver_k.find_elements(By.XPATH, "//*[@class='bp-time f-19 d-color disp-Inline']")
    total_duration = driver_k.find_elements(By.XPATH, "//*[@class='dur l-color lh-24']")
    price = driver_k.find_elements(By.XPATH, '//div[@class="fare d-block"]//span')
    seats = driver_k.find_elements(By.XPATH, "//div[contains(@class, 'seat-left')]")

    # Attempt to extract ratings (if available)
    try:
        rating = driver_k.find_elements(By.XPATH, "//div[@class='clearfix row-one']/div[@class='column-six p-right-10 w-10 fl']")
    except:
        continue  # Skip to the next link if no ratings are found

    # Append extracted details to respective lists
    for bus in bus_name:
        Bus_names_k.append(bus.text)  # Append bus name to the list
        Route_links.append(link)  # Append route link to the list
        Route_names.append(routes)  # Append route name to the list
    for bus_type_elem in bus_type:
        Bus_types_k.append(bus_type_elem.text)  # Append bus type to the list
    for start_time_elem in start_time:
        Departure_k.append(start_time_elem.text)  # Append departure time to the list
    for end_time_elem in end_time:
        Arrival_k.append(end_time_elem.text)  # Append arrival time to the list
    for total_duration_elem in total_duration:
        Total_Duration_k.append(total_duration_elem.text)  # Append total duration to the list
    for ratings_elem in rating:
        Ratings_k.append(ratings_elem.text)  # Append ratings to the list
    for price_elem in price:
        Prices_k.append(price_elem.text)  # Append price to the list
    for seats_elem in seats:
        Seats_Available_k.append(seats_elem.text)  # Append available seats to the list

# Indicate successful completion
print("Successfully Completed")


Successfully Completed


In [18]:
# Create a dictionary where each key corresponds to a column name, and the value is the list of bus details
data = {
    'Bus_name': Bus_names_k,  # List containing the names of buses
    'Bus_type': Bus_types_k,  # List containing the types of buses (e.g., Sleeper, AC, etc.)
    'Start_time': Departure_k,  # List containing the departure times of the buses
    'End_time': Arrival_k,  # List containing the arrival times of the buses
    'Total_duration': Total_Duration_k,  # List containing the total travel duration for each bus
    'Price': Prices_k,  # List containing the price of each bus ticket
    "Seats_Available": Seats_Available_k,  # List containing the number of seats available for each bus
    "Ratings": Ratings_k,  # List containing the ratings for each bus
    'Route_name': Route_names,  # List containing the name of the routes (e.g., "Kozhikode to Ernakulam")
    'Route_link': Route_links  # List containing the URL links for each route
}

# Create a pandas DataFrame from the dictionary 'data' where each key-value pair becomes a column in the DataFrame
df_wb = pd.DataFrame(data)

# Define the path where the CSV file will be saved
path = r"/Users/priyankamalavade/Desktop/redbus_scraping/scraped_all_required_bus_details/df_wb.csv"


# Convert the DataFrame 'df_wb' into a CSV file and save it at the specified path
# index=False ensures that the row index is not included in the CSV file
df_wb.to_csv(path, index=False)

# Print a message indicating that the process has been completed successfully
print("Successfully Completed")


Successfully Completed


In [19]:
df_wb

,Bus_name,Bus_type,Start_time,End_time,Total_duration,Price,Seats_Available,Ratings,Route_name,Route_link
0,WBTC (CTC) HABRA-DIGHA via Esplanade - 176|04:00,Non AC Seater (2+3),04:00,09:50,05h 50m,165,47 Seats available,4.3\n11,Digha to Barasat (West Bengal),https://www.redbus.in/bus-tickets/digha-to-bar...
1,WBTC (CTC) BARASAT-DIGHA via Bally - 38|04:45,Non AC Seater (2+3),04:45,09:35,04h 50m,161,42 Seats available,4.0\n31,Digha to Barasat (West Bengal),https://www.redbus.in/bus-tickets/digha-to-bar...
2,WBTC (CTC) BARASAT-DIGHA via Bally - 38|05:40,Non AC Seater (2+3),05:40,10:10,04h 30m,161,31 Seats available,3.6\n122,Digha to Barasat (West Bengal),https://www.redbus.in/bus-tickets/digha-to-bar...
3,WBTC (CTC) BARASAT-DIGHA via Bally - 38|07:15,Non AC Seater (2+3),07:15,11:50,04h 35m,161,31 Seats available,2.2\n12,Digha to Barasat (West Bengal),https://www.redbus.in/bus-tickets/digha-to-bar...
4,WBTC (CTC) BADURIA - DIGHA via Bally - 155|09:00,Non AC Seater (2+3),09:00,12:25,03h 25m,161,17 Seats available,2.9\n32,Digha to Barasat (West Bengal),https://www.redbus.in/bus-tickets/digha-to-bar...
...,...,...,...,...,...,...,...,...,...,...
191,,,,,,,,,Kolkata to Haldia,https://www.redbus.in/bus-tickets/kolkata-to-h...
192,,,,,,,,,Kolkata to Haldia,https://www.redbus.in/bus-tickets/kolkata-to-h...
193,,,,,,,,,Kolkata to Haldia,https://www.redbus.in/bus-tickets/kolkata-to-h...
194,,,,,,,,,Kolkata to Haldia,https://www.redbus.in/bus-tickets/kolkata-to-h...
